# Initial Setup

In [1]:
# Load Virtual Environment

# !& "c:\Users\tbran\Python\repos\Semester 3 Repos\capstone\.venv\Scripts\Activate.ps1"


In [2]:
# Initialize results list
mend_results = []
usdot_results = []

In [3]:
# --- Core ---
import os
import re
import glob
import time
import unicodedata

import numpy as np
import pandas as pd

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# --- Dates & holidays ---
import holidays

# --- Statsmodels ---
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# --- Scikit-learn: preprocessing & pipelines ---
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, OrdinalEncoder
from sklearn.pipeline import Pipeline
import category_encoders as ce
from category_encoders import TargetEncoder

# --- Scikit-learn: models ---
from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
    ElasticNet,
    LogisticRegression
)
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# --- Scikit-learn: model selection & metrics ---
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score
)
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
    mean_absolute_error,
    accuracy_score,
    balanced_accuracy_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    RocCurveDisplay,
    PrecisionRecallDisplay,
    ConfusionMatrixDisplay
)

# --- Feature selection ---
from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# --- Distributions ---
from scipy.stats import loguniform, randint

# --- Utilities ---
from sklearn.utils import resample
from sklearn.inspection import permutation_importance
from joblib import parallel_backend


In [4]:
project_path = 'C:/Users/tbran/Python/repos/Semester 3 Repos/capstone/'
data_path = project_path + 'data/'
src_path = project_path + 'src/'
model_path = project_path + 'models/'

# Mendeley Carrier Delay Dataset

## Data Prep

### Data Prep Functions

In [5]:
def optimize_dataframe(df, datetime_cols=None, fillna=False):
    """
    Cleans and optimizes a DataFrame:
    - Converts object datetime columns to datetime64
    - Converts object columns with repeated values to category
    - Downcasts numeric columns to smallest safe type
    - Optionally fills NaNs before downcasting
    
    Parameters:
        df (pd.DataFrame): The DataFrame to optimize
        datetime_cols (list): List of column names to convert to datetime
        fillna (bool): If True, fills NaNs before downcasting
    """
    
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage before optimization: {start_mem:.2f} MB")
    
    df = df.copy()
    
    # 1. Convert datetime columns
    if datetime_cols:
        for col in datetime_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # 2. Convert object columns to category if appropriate
    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        num_unique = df[col].nunique()
        num_total = len(df[col])
        if num_unique / num_total < 0.5:  # heuristic: less than 50% unique
            df[col] = df[col].astype('category')
    
    # 3. Downcast numeric columns
    int_cols = df.select_dtypes(include=['int64', 'int32']).columns
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns
    
    for col in int_cols:
        if fillna and df[col].isnull().any():
            df[col] = df[col].fillna(0)
        df[col] = pd.to_numeric(df[col], downcast='integer')
    
    for col in float_cols:
        if fillna and df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mean())
        df[col] = pd.to_numeric(df[col], downcast='float')
    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage after optimization: {end_mem:.2f} MB")
    print(f"Reduced by {100 * (start_mem - end_mem) / start_mem:.1f}%")
    
    return df


def clean_column_names(df, remove_accents=True):
    """
    Cleans DataFrame column names:
    - Strips whitespace
    - Converts to lowercase
    - Replaces spaces & special chars with underscores
    - Removes duplicate underscores
    - Optionally removes accents
    
    Parameters:
        df (pd.DataFrame): DataFrame whose columns to clean
        remove_accents (bool): If True, strips accents from characters
    
    Returns:
        pd.DataFrame: DataFrame with cleaned column names
    """
    def _clean(col):
        col = col.strip().lower()
        if remove_accents:
            col = ''.join(
                c for c in unicodedata.normalize('NFKD', col)
                if not unicodedata.combining(c)
            )
        col = re.sub(r'[^0-9a-zA-Z]+', '_', col)  # replace non-alphanumeric with _
        col = re.sub(r'_+', '_', col)             # collapse multiple underscores
        col = col.strip('_')                      # remove leading/trailing underscores
        return col
    
    df = df.copy()
    df.columns = [_clean(c) for c in df.columns]
    return df


def build_preprocessing_pipeline(df, target, 
                                  high_card_threshold=20, 
                                  scale_numeric=False):
    """
    Builds a preprocessing pipeline for linear regression:
    - One-hot encodes low-cardinality categorical columns
    - Target encodes high-cardinality categorical columns
    - Optionally scales numeric columns
    
    Parameters:
        df (pd.DataFrame): Input DataFrame (including target column)
        target (str): Name of target column
        high_card_threshold (int): Unique value cutoff for high-cardinality
        scale_numeric (bool): Whether to scale numeric features
        
    Returns:
        pipeline (ColumnTransformer): Preprocessing transformer
        low_card_cols (list): Low-cardinality categorical columns
        high_card_cols (list): High-cardinality categorical columns
        num_cols (list): Numeric columns
    """
    
    # Separate features and target
    X = df.drop(columns=[target])
    
    # Identify column types
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    
    # Split categorical into low/high cardinality
    low_card_cols = [col for col in cat_cols if X[col].nunique() <= high_card_threshold]
    high_card_cols = [col for col in cat_cols if X[col].nunique() > high_card_threshold]
    
    # Transformers
    low_card_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    high_card_transformer = TargetEncoder()
    num_transformer = StandardScaler() if scale_numeric else 'passthrough'
    
    # Column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('low_card', low_card_transformer, low_card_cols),
            ('high_card', high_card_transformer, high_card_cols),
            ('num', num_transformer, num_cols)
        ]
    )
    
    return preprocessor, low_card_cols, high_card_cols, num_cols

def add_interaction_terms(df, features):
    """
    Adds pairwise interaction terms between given features.
    """
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    interaction_array = poly.fit_transform(df[features])
    interaction_df = pd.DataFrame(interaction_array, columns=poly.get_feature_names_out(features))
    return pd.concat([df.reset_index(drop=True), interaction_df], axis=1)

def preprocess_features(df, categorical_cols, numeric_cols):
    """
    Returns a ColumnTransformer that one-hot encodes categorical columns
    and passes numeric columns through unchanged.
    """
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
            ('num', 'passthrough', numeric_cols)
        ]
    )
    return preprocessor

def transform_with_names(preprocessor, X, y=None):
    """Fit/transform and return a DataFrame with feature names preserved."""
    Xt = preprocessor.fit_transform(X, y)
    cols = preprocessor.get_feature_names_out()
    return pd.DataFrame(Xt, columns=cols, index=X.index)


def create_features_mend(df):
    df = df.copy()

    # Build U.S. holiday calendar for relevant years
    us_holidays = holidays.US(years=range(2015, 2027))

    # Build a list of (holiday_date, holiday_name)
    holiday_items = list(us_holidays.items())

    # Create a set of all holiday dates ±3 days
    holiday_buffer = {}
    for h_date, h_name in holiday_items:
        for offset in range(-3, 4):  # -3, -2, -1, 0, +1, +2, +3
            holiday_buffer[h_date + pd.Timedelta(days=offset)] = h_name

    # Binary flag: within ±3 days of a holiday
    if "scheduleddepartdatetime" in df.columns:
        print("Adding holiday features...")
        df['is_holiday_period'] = df['scheduleddepartdatetime'].dt.date.isin(holiday_buffer.keys())

        # Categorical holiday name (or "None")
        def get_holiday_name(d):
            return holiday_buffer.get(d, "None")

        df['holiday_name'] = df['scheduleddepartdatetime'].dt.date.apply(get_holiday_name)
    else:
        print("Column 'scheduleddepartdatetime' not found, skipping holiday features.")

    # --- Existing engineered features ---
    if "scheduleddepartdatetime" in df.columns:
        print("Adding time-based features...")
        df["dayofweek"] = df["scheduleddepartdatetime"].dt.dayofweek
        df["month"] = df["scheduleddepartdatetime"].dt.month
    else:
        print("Skipping time-based features.")

    if {"origin","dest"}.issubset(df.columns):
        print("Adding route feature...")
        df["route"] = df["origin"].astype(str) + "_" + df["dest"].astype(str)
    else:
        print("Skipping route feature.")

    if {"marketshareorigin","marketsharedest"}.issubset(df.columns):
        print("Adding marketshare_diff...")
        df["marketshare_diff"] = df["marketshareorigin"] - df["marketsharedest"]
    else:
        print("Skipping marketshare_diff.")

    if {"hhiorigin","hhidest"}.issubset(df.columns):
        print("Adding hhi_diff...")
        df["hhi_diff"] = df["hhiorigin"] - df["hhidest"]
    else:
        print("Skipping hhi_diff.")

    if {"temperature","windspeed"}.issubset(df.columns):
        print("Adding temp_wind_interaction...")
        df["temp_wind_interaction"] = df["temperature"] * df["windspeed"]
    else:
        print("Skipping temp_wind_interaction.")

    if {"temperature","windgustspeed"}.issubset(df.columns):
        print("Adding temp_windgust_interaction...")
        df["temp_windgust_interaction"] = df["temperature"] * df["windgustspeed"]
    else:
        print("Skipping temp_windgust_interaction.")

    if {"windspeed","windgustspeed"}.issubset(df.columns):
        print("Adding wind_gust_diff...")
        df["wind_gust_diff"] = df["windspeed"] - df["windgustspeed"]
    else:
        print("Skipping wind_gust_diff.")

    if {"raindummy","windspeed"}.issubset(df.columns):
        print("Adding rain_wind_interaction...")
        df["rain_wind_interaction"] = df["raindummy"] * df["windspeed"]
    else:
        print("Skipping rain_wind_interaction.")

    if {"snowdummy","windspeed"}.issubset(df.columns):
        print("Adding snow_wind_interaction...")
        df["snow_wind_interaction"] = df["snowdummy"] * df["windspeed"]
    else:
        print("Skipping snow_wind_interaction.")

    if {"raindummy","windgustspeed"}.issubset(df.columns):
        print("Adding rain_wind_gust_interaction...")
        df["rain_wind_gust_interaction"] = df["raindummy"] * df["windgustspeed"]
    else:
        print("Skipping rain_wind_gust_interaction.")

    if {"snowdummy","windgustspeed"}.issubset(df.columns):
        print("Adding snow_wind_gust_interaction...")
        df["snow_wind_gust_interaction"] = df["snowdummy"] * df["windgustspeed"]
    else:
        print("Skipping snow_wind_gust_interaction.")

    if {"originmetropop","destmetropop"}.issubset(df.columns):
        print("Adding metropop_diff...")
        df["metropop_diff"] = df["originmetropop"] - df["destmetropop"]
    else:
        print("Skipping metropop_diff.")

    if {"originmetrogdppercapita","destmetrogdppercapita"}.issubset(df.columns):
        print("Adding metrogdp_diff...")
        df["metrogdp_diff"] = df["originmetrogdppercapita"] - df["destmetrogdppercapita"]
    else:
        print("Skipping metrogdp_diff.")

    return df

def engineer_flight_features_light(
    df,
    datetime_col="scheduleddepartdatetime",
    origin_col="origin",
    dest_col="dest",
    carrier_col="uniquecarrier",
    delay_col="depdelay",
    distance_col="distance"
):
    df = df.copy()

    # --- Datetime parts ---
    dt = pd.to_datetime(df[datetime_col])
    df["year"] = dt.dt.year
    df["month"] = dt.dt.month
    df["day"] = dt.dt.day
    df["hour"] = dt.dt.hour
    df["date"] = dt.dt.floor("D")
    df["is_weekend"] = dt.dt.dayofweek >= 5

    # --- Route & distance ---
    df["route"] = df[origin_col].astype(str) + "_" + df[dest_col].astype(str)
    if distance_col in df.columns:
        df["distance_bin"] = pd.cut(
            df[distance_col],
            bins=[0, 500, 1500, 3000, 10000],
            labels=["short", "medium", "long", "ultra"]
        )

    # --- Congestion (lighter via merge instead of transform) ---
    # Hourly origin flights
    hourly_counts = (
        df.groupby([origin_col, "date", "hour"], observed=True)
          .size()
          .reset_index(name="hourly_origin_flights")
    )
    df = df.merge(hourly_counts, on=[origin_col, "date", "hour"], how="left")

    # Daily route flights
    daily_counts = (
        df.groupby([origin_col, dest_col, "date"], observed=True)
          .size()
          .reset_index(name="daily_route_flights")
    )
    df = df.merge(daily_counts, on=[origin_col, dest_col, "date"], how="left")

    # --- Weather (light) ---
    if "temperature" in df.columns:
        df["is_extreme_temp"] = (df["temperature"] < 0) | (df["temperature"] > 35)
    if {"raindummy", "snowdummy", "windgustdummy"}.issubset(df.columns):
        df["stormy"] = (df["raindummy"] | df["snowdummy"] | df["windgustdummy"]).astype(int)

    # --- Market/demand ---
    if {"capacity", "numflights"}.issubset(df.columns):
        df["capacity_utilization"] = df["numflights"] / df["capacity"].replace(0, np.nan)

    return df


def engineer_flight_features_heavy(
    df,
    datetime_col="scheduleddepartdatetime",
    origin_col="origin",
    dest_col="dest",
    carrier_col="uniquecarrier",
    delay_col="depdelay",
    distance_col="distance",
    window=7
):
    """
    Engineer advanced features for flight delay prediction.
    Optimized for speed, with print statements and timing checkpoints.
    """

    start_time = time.time()
    df = df.copy()
    print("Starting feature engineering...")

    # --- Precompute datetime parts once ---
    t0 = time.time()
    if datetime_col in df.columns:
        dt = pd.to_datetime(df[datetime_col])
        df["year"] = dt.dt.year
        df["month"] = dt.dt.month
        df["day"] = dt.dt.day
        df["hour"] = dt.dt.hour
        df["date"] = dt.dt.floor("D")
        df["quarter"] = dt.dt.quarter
        df["is_weekend"] = dt.dt.dayofweek >= 5
        df["part_of_day"] = pd.cut(
            df["hour"],
            bins=[0,5,11,16,21,24],
            labels=["late_night","morning","midday","evening","night"],
            right=False
        )
        df["days_since_year_start"] = (
            dt - pd.to_datetime(df["year"].astype(str) + "-01-01")
        ).dt.days
    print(f"Datetime features done in {time.time()-t0:.2f}s")

    # --- Route & distance features ---
    t0 = time.time()
    if {origin_col, dest_col}.issubset(df.columns):
        if f"largehubairport{origin_col}" in df.columns and f"largehubairport{dest_col}" in df.columns:
            df["hub_to_hub"] = (
                (df[f"largehubairport{origin_col}"] == 1) &
                (df[f"largehubairport{dest_col}"] == 1)
            ).astype(int)
        df["route"] = df[origin_col].astype(str) + "_" + df[dest_col].astype(str)

    if distance_col in df.columns:
        df["distance_bin"] = pd.cut(
            df[distance_col],
            bins=[0,500,1500,3000,10000],
            labels=["short","medium","long","ultra"]
        )
    print(f"Route & distance features done in {time.time()-t0:.2f}s")

    # --- Congestion features ---
    t0 = time.time()
    if {"date","hour",origin_col}.issubset(df.columns):
        hourly_counts = (
            df.groupby([origin_col,"date","hour"])
              .size()
              .rename("hourly_origin_flights")
              .reset_index()
        )
        df = df.merge(hourly_counts, on=[origin_col,"date","hour"], how="left")

    if {"date",origin_col,dest_col}.issubset(df.columns):
        daily_counts = (
            df.groupby([origin_col,dest_col,"date"])
              .size()
              .rename("daily_route_flights")
              .reset_index()
        )
        df = df.merge(daily_counts, on=[origin_col,dest_col,"date"], how="left")
    print(f"Congestion features done in {time.time()-t0:.2f}s")

    # --- Weather features ---
    t0 = time.time()
    if "temperature" in df.columns:
        df["is_extreme_temp"] = (df["temperature"] < 0) | (df["temperature"] > 35)
        if "month" in df.columns:
            monthly_means = df.groupby("month")["temperature"].transform("mean")
            df["temp_anomaly"] = df["temperature"] - monthly_means

    if {"raindummy","snowdummy","windgustdummy"}.issubset(df.columns):
        df["stormy"] = (
            (df["raindummy"]==1) | (df["snowdummy"]==1) | (df["windgustdummy"]==1)
        ).astype(int)
    print(f"Weather features done in {time.time()-t0:.2f}s")

    # --- Rolling averages ---
    t0 = time.time()
    if {origin_col, delay_col}.issubset(df.columns):
        df = df.sort_values([origin_col, datetime_col])
        df["rolling_origin_delay"] = (
            df.groupby(origin_col)[delay_col]
              .rolling(window, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )

    if {dest_col, delay_col}.issubset(df.columns):
        df = df.sort_values([dest_col, datetime_col])
        df["rolling_dest_delay"] = (
            df.groupby(dest_col)[delay_col]
              .rolling(window, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )

    if {carrier_col, delay_col}.issubset(df.columns):
        df = df.sort_values([carrier_col, datetime_col])
        df["rolling_carrier_delay"] = (
            df.groupby(carrier_col)[delay_col]
              .rolling(window, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )

    if {"route", delay_col}.issubset(df.columns):
        df = df.sort_values(["route", datetime_col])
        df["rolling_route_delay"] = (
            df.groupby("route")[delay_col]
              .rolling(window, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )
    print(f"Rolling averages done in {time.time()-t0:.2f}s")

    # --- Market/demand features ---
    t0 = time.time()
    if {"capacity","numflights"}.issubset(df.columns):
        df["capacity_utilization"] = (
            df["numflights"] / df["capacity"].replace(0, np.nan)
        )

    if {origin_col, dest_col, carrier_col}.issubset(df.columns):
        df["route_carrier_count"] = (
            df.groupby([origin_col,dest_col])[carrier_col].transform("nunique")
        )
    print(f"Market/demand features done in {time.time()-t0:.2f}s")

    # --- Interaction features ---
    t0 = time.time()
    if {"is_holiday_period","monopolyroute"}.issubset(df.columns):
        df["holiday_monopoly"] = (
            df["is_holiday_period"].astype(int) * df["monopolyroute"].astype(int)
        )

    if {"is_extreme_temp","hourly_origin_flights"}.issubset(df.columns):
        df["extreme_temp_congestion"] = (
            df["is_extreme_temp"].astype(int) * df["hourly_origin_flights"]
        )
    print(f"Interaction features done in {time.time()-t0:.2f}s")

    print(f"Total feature engineering time: {time.time()-start_time:.2f}s")
    return df



### Data Prep Steps

In [6]:
# Load Mendeley Delay Data
file_name = 'MendeleyDelayData.csv'
df_mend = pd.read_csv(data_path + file_name)

df_mend = optimize_dataframe(
    df_mend,
    datetime_cols=['scheduleddepartdatetime'],
    fillna=True
)
df_mend = clean_column_names(df_mend)

Memory usage before optimization: 1008.24 MB


Memory usage after optimization: 150.66 MB
Reduced by 85.1%


In [7]:
file_name = "FAA_AC_REGISTRATION_2021.csv"
df_reg = pd.read_csv(data_path + file_name)

# Left join on stripped tail_num
df_mend_clean_reg = df_mend.merge(
    df_reg,
    how="left",
    left_on=df_mend["tailnum"].str.lstrip("N"),
    right_on="N-NUMBER"
)

# Impute missing values with mode per column
mode_dict = {col: df_reg[col].mode()[0] for col in df_reg.columns if col != "N-NUMBER"}
df_mend_clean_reg = df_mend_clean_reg.fillna(mode_dict)

# Drop duplicate join key
df_mend_clean_reg = df_mend_clean_reg.drop(columns=["N-NUMBER"])
df_mend_clean_reg = clean_column_names(df_mend_clean_reg)

C:\Users\tbran\AppData\Local\Temp\ipykernel_5396\3905881234.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reg = pd.read_csv(data_path + file_name)


In [8]:
# Define encoders at module scope (picklable)
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=np.int8)
te = TargetEncoder()
ord_enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

def build_dual_preprocessors(df, target, feature_cols,
                             high_card_threshold=20, scale_numeric=False):
    """
    Build regression + tree preprocessors using an explicit feature list.
    Returns (regression_preprocessor, tree_preprocessor, X, y).
    """
    X = df[feature_cols].copy()
    y = df[target]

    # Identify column types
    cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
    num_cols = X.select_dtypes(include=["number"]).columns.tolist()

    # Split categorical into low/high cardinality
    low_card_cols = [c for c in cat_cols if X[c].nunique() <= high_card_threshold]
    high_card_cols = [c for c in cat_cols if X[c].nunique() > high_card_threshold]

    # Regression preprocessor: OHE + TargetEncoder + scaling
    regression_preprocessor = ColumnTransformer(
        transformers=[
            ("low_card", ohe, low_card_cols),
            ("high_card", te, high_card_cols),
            ("num", StandardScaler() if scale_numeric else "passthrough", num_cols),
        ]
    )

    # Tree preprocessor: Ordinal encode categoricals, passthrough numerics
    tree_preprocessor = ColumnTransformer(
        transformers=[
            ("cat", ord_enc, cat_cols),
            ("num", "passthrough", num_cols),
        ]
    )

    return regression_preprocessor, tree_preprocessor, X, y

In [9]:
# Remove outliers from dataframe
df_mend_clean = df_mend_clean_reg[df_mend_clean_reg['depdelay'] >= -30]

# Create engineered features
df_mend_clean = create_features_mend(df_mend_clean)
df_mend_clean = engineer_flight_features_light(df_mend_clean)

# Get list of columns
df_mend_id_cols = ['originairportid', 'destairportid', ]
df_mend_cat_cols = ['origin', 'dest', 'uniquecarrier', 'tailnum', 'origincityname', 'originstate', ]
df_mend_date_cols = ['scheduleddepartdatetime', ]
df_mend_target_cols = ['depdelay','arrdelay',]
df_mend_feature_cols = [col for col in df_mend_clean.columns if col not in df_mend_id_cols + df_mend_cat_cols + df_mend_date_cols + df_mend_target_cols]

# drop leakage columns, ID columns, and date columns
df_mend_clean = df_mend_clean.drop(columns=['arrdelay'] + df_mend_id_cols + df_mend_date_cols).copy()
# Export df to csv
df_mend_clean.to_csv(data_path + 'MendeleyDelayData_Cleaned.csv', index=False)

reg_prep_mend, tree_prep_mend, X_mend, y_mend_numeric = build_dual_preprocessors(df_mend_clean, 
                                                                                 target='depdelay', 
                                                                                 feature_cols=df_mend_feature_cols, 
                                                                                 high_card_threshold=30, scale_numeric=True)

# Create binary target for classification (15 min delay threshold)
y_mend_binary_15 = (y_mend_numeric >= 15).astype(int)

# Ensure categorical columns are string type (Some reg data has mixed types)
cat_cols = ['type_acft', 'ac_weight', 'holiday_name', 'distance_bin']
high_card_cols = ['mfr', 'model', 'route']

for col in cat_cols + high_card_cols:
    X_mend[col] = X_mend[col].astype(str)

# Create seperate transformed datasets for regression and tree models
X_reg_mend = transform_with_names(reg_prep_mend, X_mend, y_mend_numeric)
X_tree_mend = transform_with_names(tree_prep_mend, X_mend, y_mend_numeric)


Adding holiday features...


Adding time-based features...
Adding route feature...


Adding marketshare_diff...
Adding hhi_diff...
Adding temp_wind_interaction...
Adding temp_windgust_interaction...
Adding wind_gust_diff...
Adding rain_wind_interaction...
Adding snow_wind_interaction...
Adding rain_wind_gust_interaction...
Adding snow_wind_gust_interaction...
Adding metropop_diff...
Adding metrogdp_diff...


## Week 1 – Linear Regression 1
Each week, you will apply the concepts of that week to your Integrated Capstone Project’s dataset. In preparation for Milestone One, create a Jupyter Notebook (similar to in Module B, semester two) that illustrates these lessons. There are no specific questions to answer in your Jupyter Notebook files in this course; your general goal is to analyze your data, using the methods you have learned about in this course and in this program, and draw interesting conclusions. 

For Week 1, include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday. 

### Week 1 Helper Functions

In [10]:
def regression_summary(X, y):
    """
    Fits an OLS regression model using statsmodels and prints the summary.
    """
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    return model.summary()

def fit_polynomial_regression(X, y, degree=2):
    """
    Fits a polynomial regression model and returns the fitted model and transformed features.
    """
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, y)
    return model, poly


def calculate_vif(df, features=None, vif_thresh=10.0):
    """
    Calculate Variance Inflation Factor (VIF) safely:
    - Removes constant and perfectly collinear columns
    - Returns a clean, sorted VIF table
    """
    if features is None:
        features = df.select_dtypes(include=[np.number]).columns.tolist()
    
    X = df[features].copy()

    # 1. Drop constant columns
    constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
    if constant_cols:
        print(f"Dropping constant columns: {constant_cols}")
        X.drop(columns=constant_cols, inplace=True)

    # 2. Drop perfectly collinear columns
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    perfect_corr_cols = [col for col in upper.columns if any(upper[col] == 1.0)]
    if perfect_corr_cols:
        print(f"Dropping perfectly collinear columns: {perfect_corr_cols}")
        X.drop(columns=perfect_corr_cols, inplace=True)

    # 3. Calculate VIF
    X_const = sm.add_constant(X)  # cleaner than assign(const=1)
    vif_data = pd.DataFrame({
        "Feature": X.columns,
        "VIF": [variance_inflation_factor(X_const.values, i+1)  # skip const
                for i in range(len(X.columns))]
    })

    # 4. Sort and format
    vif_data["VIF"] = vif_data["VIF"].round(2)
    vif_data["High_VIF"] = vif_data["VIF"] > vif_thresh
    vif_data = vif_data.sort_values(by="VIF", ascending=False).reset_index(drop=True)

    return vif_data

### Linear Regression

In [11]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_mend, y_mend_numeric, test_size=0.2, random_state=42
)

# Build pipeline
linreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", LinearRegression())
])

# Fit
linreg_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = linreg_pipe.predict(X_train)
y_pred_test = linreg_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

# Store results
mend_results.append({
    'model': 'Linear Regression',
    'train_r2': train_r2,
    'test_r2': test_r2, 
    'test_rmse': test_rmse
})

Train R²: 0.056
Test R²: 0.047
Test RMSE: 34.318


### Polynomial Regression: Second Attempt

Reduced the number of features

In [12]:
# Choose 5 features explicitly
selected_features = ["raindummy","snowdummy","windgustdummy","temperature","windspeed","originmetropop", "originmetrogdppercapita"]

# Reduce dataset to 20% of original, but only keep those 5 columns
X_small, _, y_small, _ = train_test_split(
    X_mend[selected_features],  # <--- subset here
    y_mend_numeric,
    train_size=0.2,
    random_state=42
)


# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)


# Define a new preprocessor for just these features
reg_prep_small = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), selected_features)
    ]
)

poly_reg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_small),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("model", LinearRegression())
])

# Fit
poly_reg_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = poly_reg_pipe.predict(X_train)
y_pred_test = poly_reg_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

# Store results
mend_results.append({
    'model': 'Polynomial Regression',
    'train_r2': train_r2,
    'test_r2': test_r2,
    'test_rmse': test_rmse
})

Train R²: 0.018
Test R²: 0.017
Test RMSE: 34.597


### VIF: Variable Inflation Factor

In [13]:
X_reg_mend_sample = X_reg_mend.sample(n=50000, random_state=42) if len(X_reg_mend) > 10000 else X_reg_mend
vif_table = calculate_vif(X_reg_mend_sample, features=X_reg_mend_sample.columns.tolist(), vif_thresh=10.0)
print(vif_table)

Dropping constant columns: ['low_card__type_acft_7']


Dropping perfectly collinear columns: ['num__day', 'num__hour']


c:\Users\tbran\Python\repos\Semester 3 Repos\capstone\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                        Feature   VIF  High_VIF
0         low_card__type_acft_4   inf      True
1         low_card__type_acft_5   inf      True
2         low_card__type_acft_6   inf      True
3   low_card__ac_weight_CLASS 1   inf      True
4   low_card__ac_weight_CLASS 4   inf      True
..                          ...   ...       ...
95           num__scheduledhour  1.07     False
96     num__daily_route_flights  1.05     False
97              num__dayofmonth  1.04     False
98          num__raintracedummy  1.02     False
99                  num__ac_cat  1.01     False

[100 rows x 3 columns]


## Week 2 - Linear Regression 2

For Week 2, include concepts such as linear regression with lasso, ridge, and elastic net regression. This homework will be submitted for peer review and feedback in Week 3 in the assignment titled 3.4 Peer Review: Week 2 Jupyter Notebook. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday.

### Lasso Regression

In [14]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Build pipeline with preprocessing + Lasso
lasso_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", Lasso(alpha=0.1, max_iter=10000, random_state=42))
])

# Fit
lasso_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = lasso_pipe.predict(X_train)
y_pred_test = lasso_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

#store results
mend_results.append({
    'model': 'Lasso Regression',
    'train_r2': train_r2,
    'test_r2': test_r2,
    'test_rmse': test_rmse
})  

Train R²: 0.066
Test R²: 0.036
Test RMSE: 34.256


### Lasso Grid Search

In [15]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Pipeline: preprocessing + Lasso
lasso_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", Lasso(max_iter=10000, random_state=42))
])

# Grid of hyperparameters to search
param_grid = {
    "model__alpha": [0.001, 0.01, 0.1, 1, 10]
}

# Grid search with 5-fold CV
grid = GridSearchCV(
    lasso_pipe,
    param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit grid search
grid.fit(X_train, y_train)

# Best parameters
print("Best alpha:", grid.best_params_["model__alpha"])

# Evaluate on test set
y_pred_test = grid.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

# Store results
mend_results.append({
    'model': 'Lasso Regression (Tuned)',
    'best_alpha': grid.best_params_["model__alpha"],
    'train_r2': grid.best_score_,
    'test_r2': test_r2,
    'test_rmse': test_rmse
})

Best alpha: 0.001
Test R²: 0.037
Test RMSE: 34.246


### Ridge Regression

In [16]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Build pipeline with preprocessing + Ridge
ridge_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", Ridge(alpha=1.0, max_iter=10000, random_state=42))
])

# Fit
ridge_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = ridge_pipe.predict(X_train)
y_pred_test = ridge_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

#store results
mend_results.append({
    'model': 'Ridge Regression',
    'train_r2': train_r2,
    'test_r2': test_r2, 
    'test_rmse': test_rmse
})

Train R²: 0.067
Test R²: 0.037
Test RMSE: 34.246


### Ridge Grid Search

In [17]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)
# Pipeline: preprocessing + Ridge
ridge_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", Ridge(max_iter=10000, random_state=42))
])

# Grid of hyperparameters
ridge_param_grid = {
    "model__alpha": [0.01, 0.1, 1, 10, 100]
}

ridge_grid = GridSearchCV(
    ridge_pipe,
    ridge_param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

ridge_grid.fit(X_train, y_train)

print("Best Ridge alpha:", ridge_grid.best_params_["model__alpha"])

y_pred_test = ridge_grid.predict(X_test)
print("Ridge Test R²:", r2_score(y_test, y_pred_test))
print("Ridge Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))

#store results
mend_results.append({
    'model': 'Ridge Regression (Tuned)',
    'best_alpha': ridge_grid.best_params_["model__alpha"],
    'train_r2': ridge_grid.best_score_,
    'test_r2': r2_score(y_test, y_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test))
})

Best Ridge alpha: 100
Ridge Test R²: 0.03657956549522978
Ridge Test RMSE: 34.24586867799605


### Elastic Net

In [18]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Build pipeline with preprocessing + Elastic Net
elasticnet_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000, random_state=42))
])
# Fit
elasticnet_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = elasticnet_pipe.predict(X_train)
y_pred_test = elasticnet_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

#store results
mend_results.append({
    'model': 'Elastic Net Regression',
    'train_r2': train_r2,
    'test_r2': test_r2, 
    'test_rmse': test_rmse
})

Train R²: 0.066
Test R²: 0.036
Test RMSE: 34.259


### Elastic Net Grid Search

In [19]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)
# Pipeline: preprocessing + ElasticNet
elastic_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", ElasticNet(max_iter=10000, random_state=42))
])

# Grid of hyperparameters
elastic_param_grid = {
    "model__alpha": [0.001, 0.01, 1, 10],
    "model__l1_ratio": [0.2,  0.8]  # balance between L1 and L2
}

elastic_grid = GridSearchCV(
    elastic_pipe,
    elastic_param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

elastic_grid.fit(X_train, y_train)

print("Best ElasticNet params:", elastic_grid.best_params_)

y_pred_test = elastic_grid.predict(X_test)
print("ElasticNet Test R²:", r2_score(y_test, y_pred_test))
print("ElasticNet Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))

#store results
mend_results.append({
    'model': 'Elastic Net Regression (Tuned)',
    'best_params': elastic_grid.best_params_,
    'train_r2': elastic_grid.best_score_,
    'test_r2': r2_score(y_test, y_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test))
})

Best ElasticNet params: {'model__alpha': 0.001, 'model__l1_ratio': 0.8}
ElasticNet Test R²: 0.036592362709670034
ElasticNet Test RMSE: 34.24564123151225


## Week 3 - Linear Regression 3

For Week 3, include concepts such as linear regression with forward and backward selection, PCR, and PLSR. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday. 

### Forward & Backward Selection: Linear Regression

In [ ]:
# --- Shrink dataset to 20% ---
X_sample, _, y_sample, _ = train_test_split(
    X_mend, y_mend_numeric,
    test_size=0.8,
    random_state=42
)

# Preprocess once
X_proc = reg_prep_mend.fit_transform(X_sample, y_sample)
y = y_sample

# --- Forward Selection (limit to 30 features, step=2) ---
forward_rmse = []
k_range_fwd = range(1, min(31, X_proc.shape[1] + 1), 2)

for k in k_range_fwd:
    sfs = SequentialFeatureSelector(
        estimator=LinearRegression(),
        n_features_to_select=k,
        direction="forward",
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    sfs.fit(X_proc, y)
    mask = sfs.get_support()
    # Evaluate using CV on the reduced feature set
    scores = cross_val_score(
        LinearRegression(),
        X_proc[:, mask],
        y,
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    rmse = np.sqrt(-scores.mean())
    forward_rmse.append(rmse)

best_idx_fwd = np.argmin(forward_rmse)
best_k_fwd = list(k_range_fwd)[best_idx_fwd]
best_rmse_fwd = forward_rmse[best_idx_fwd]

mend_results.append({
    'model': 'Forward Selection Linear Regression',
    'num_features': best_k_fwd,
    'best_cv_rmse': best_rmse_fwd,
    'rmse_curve': forward_rmse,
    'k_range': list(k_range_fwd)
})

# --- Backward Selection (step size=3, early stopping) ---
backward_rmse = []
k_range_bwd = range(1, X_proc.shape[1] + 1, 3)

tolerance = 1e-3
best_rmse_so_far = np.inf

for k in k_range_bwd:
    sbs = SequentialFeatureSelector(
        estimator=LinearRegression(),
        n_features_to_select=k,
        direction="backward",
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    sbs.fit(X_proc, y)
    mask = sbs.get_support()

    # Evaluate this subset with CV
    scores = cross_val_score(
        LinearRegression(),
        X_proc[:, mask],
        y,
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    rmse = np.sqrt(-scores.mean())
    backward_rmse.append(rmse)

    # Early stopping if curve flattens
    if best_rmse_so_far - rmse < tolerance:
        print(f"Stopping early at {k} features (no significant improvement).")
        break
    best_rmse_so_far = min(best_rmse_so_far, rmse)


best_idx_bwd = np.argmin(backward_rmse)
best_k_bwd = list(k_range_bwd)[:len(backward_rmse)][best_idx_bwd]
best_rmse_bwd = backward_rmse[best_idx_bwd]

mend_results.append({
    'model': 'Backward Selection Linear Regression',
    'num_features': best_k_bwd,
    'best_cv_rmse': best_rmse_bwd,
    'rmse_curve': backward_rmse,
    'k_range': list(k_range_bwd)[:len(backward_rmse)]
})

# --- Plot curves ---
plt.figure(figsize=(8,5))
plt.plot(k_range_fwd, forward_rmse, marker="o", label="Forward Selection")
plt.plot(list(k_range_bwd)[:len(backward_rmse)], backward_rmse, marker="s", label="Backward Selection")
plt.xlabel("Number of Features Selected")
plt.ylabel("CV RMSE")
plt.title("Forward vs Backward Selection (optimized)")
plt.legend()
plt.grid(True)
plt.show()

### PCR
Principal Component Regression

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# PCR pipeline: preprocessing → PCA → Linear Regression
pcr_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),   # your ColumnTransformer
    ("pca", PCA()),                    # dimensionality reduction
    ("model", LinearRegression())
])

# Grid search over number of components
max_components = min(X_train.shape[0], X_train.shape[1])

param_grid = {
    "pca__n_components": list(range(5, max_components+1, 5))
}

pcr_grid = GridSearchCV(
    pcr_pipe,
    param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit
pcr_grid.fit(X_train, y_train)

# Best number of components
print("Best n_components:", pcr_grid.best_params_["pca__n_components"])

# Evaluate on test set
y_pred_test = pcr_grid.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"PCR Test R²: {test_r2:.3f}")
print(f"PCR Test RMSE: {test_rmse:.3f}")

# Store results for comparison
mend_results.append({
    "Model": "PCR",
    "Best Params": pcr_grid.best_params_,
    "Test R²": test_r2,
    "Test RMSE": test_rmse
})


### PLSR
Partial Least Squares Regression

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_mend, y_mend_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_mend_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# PLSR pipeline: preprocessing → PLSRegression
pls_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),   # your ColumnTransformer
    ("model", PLSRegression())
])

# Grid search over number of components
param_grid = {
    "model__n_components": [2, 5, 10, 20, 40]  # tune based on dataset size
}

pls_grid = GridSearchCV(
    pls_pipe,
    param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit
pls_grid.fit(X_train, y_train)

# Best number of components
print("Best n_components:", pls_grid.best_params_["model__n_components"])

# Evaluate on test set
y_pred_test = pls_grid.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"PLSR Test R²: {test_r2:.3f}")
print(f"PLSR Test RMSE: {test_rmse:.3f}")

# Store results for comparison
mend_results.append({
    "Model": "PLSR",
    "Best Params": pls_grid.best_params_,
    "Test R²": test_r2,
    "Test RMSE": test_rmse
})

## Week 4 - Logistic Regression and Feature Scaling

For Week 4, include concepts such as logistic regression and feature scaling. This homework should be submitted for peer review in the assignment titled 4.3 Peer Review: Week 4 Jupyter Notebook. Complete and submit your Jupyter Notebook homework by 11:59pm ET on Sunday. 

### Log Regression: Basic

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)


# Build pipeline: preprocessing + logistic regression
logreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", LogisticRegression(max_iter=1000, solver="liblinear"))
])

# Fit
logreg_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = logreg_pipe.predict(X_train)
y_pred_test = logreg_pipe.predict(X_test)
y_pred_proba = logreg_pipe.predict_proba(X_test)[:, 1]

# Metrics
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
train_bal_acc = balanced_accuracy_score(y_train, y_pred_train)
test_bal_acc = balanced_accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Train Accuracy: {train_acc:.3f}")
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Train Balanced Accuracy: {train_bal_acc:.3f}")
print(f"Test Balanced Accuracy: {test_bal_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
mend_results.append({
    "Model": "Logistic Regression",
    "Train Accuracy": train_acc,
    "Test Accuracy": test_acc,
    "Train Balanced Accuracy": train_bal_acc,
    "Test Balanced Accuracy": test_bal_acc,

    "Test AUC": test_auc
})

### Log Regression: Forward & Backward Selection: Light

In [ ]:
# --- Shrink dataset to 20% with stratification on binary target ---
X_sample, _, y_sample, _ = train_test_split(
    X_mend, y_mend_binary_15,
    test_size=0.8,
    random_state=42,
    stratify=y_mend_binary_15
)

# Preprocess once
X_proc = reg_prep_mend.fit_transform(X_sample)
y = y_sample

# Define estimator
logreg = LogisticRegression(max_iter=1000, solver="liblinear")

# --- Forward Selection (limit to 30 features) ---
forward_scores = []
k_range_fwd = range(1, min(41, X_proc.shape[1] + 1))

tolerance = 1e-3
best_score_so_far = -np.inf

for k in k_range_fwd:
    sfs = SequentialFeatureSelector(
        estimator=logreg,
        n_features_to_select=k,
        direction="forward",
        scoring="balanced_accuracy",
        cv=3,
        n_jobs=-1
    )
    sfs.fit(X_proc, y)
    mask = sfs.get_support()
    score = cross_val_score(
        logreg,
        X_proc[:, mask],
        y,
        scoring="balanced_accuracy",
        cv=3
    ).mean()
    forward_scores.append(score)

    # Early stopping if improvement is too small
    if score - best_score_so_far < tolerance:
        print(f"Forward selection stopping early at {k} features (Δ<{tolerance}).")
        break
    best_score_so_far = max(best_score_so_far, score)

# Best forward
best_idx_fwd = np.argmax(forward_scores)
best_k_fwd = list(k_range_fwd)[best_idx_fwd]
best_score_fwd = forward_scores[best_idx_fwd]

mend_results.append({
    'model': 'Forward Selection Logistic Regression',
    'num_features': best_k_fwd,
    'best_cv_bal_acc': best_score_fwd,
    'score_curve': forward_scores,
    'k_range': list(k_range_fwd)
})

# --- Backward Selection (step size = 3, early stopping) ---
backward_scores = []
n_features = X_proc.shape[1]

tolerance = 1e-3
best_score_so_far = -np.inf

for k in range(n_features, 0, -1):  # from all features down to 1
    sbs = SequentialFeatureSelector(
        estimator=logreg,
        n_features_to_select=k,
        direction="backward",
        scoring="balanced_accuracy",
        cv=3,
        n_jobs=-1
    )
    sbs.fit(X_proc, y)
    mask = sbs.get_support()
    score = cross_val_score(
        logreg,
        X_proc[:, mask],
        y,
        scoring="balanced_accuracy",
        cv=3
    ).mean()
    backward_scores.append(score)

    # Early stopping if curve flattens
    if score - best_score_so_far < tolerance:
        print(f"Backward selection stopping early at {k} features (Δ<{tolerance}).")
        break
    best_score_so_far = max(best_score_so_far, score)

# Reverse scores so they align with decreasing k
backward_scores = backward_scores[::-1]
k_range_bwd = list(range(n_features, n_features - len(backward_scores), -1))[::-1]

# Best backward
best_idx_bwd = np.argmax(backward_scores)
best_k_bwd = list(k_range_bwd)[:len(backward_scores)][best_idx_bwd]
best_score_bwd = backward_scores[best_idx_bwd]

mend_results.append({
    'model': 'Backward Selection Logistic Regression',
    'num_features': best_k_bwd,
    'best_cv_bal_acc': best_score_bwd,
    'score_curve': backward_scores,
    'k_range': list(k_range_bwd)[:len(backward_scores)]
})

# --- Plot curves ---
plt.figure(figsize=(8,5))
plt.plot(k_range_fwd, forward_scores, marker="o", label="Forward Selection")
plt.plot(list(k_range_bwd)[:len(backward_scores)], backward_scores, marker="s", label="Backward Selection")
plt.xlabel("Number of Features Selected")
plt.ylabel("CV Balanced Accuracy")
plt.title("Forward vs Backward Selection (Logistic Regression)")
plt.legend()
plt.grid(True)
plt.show()


### Log Regression: Random Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

# Pipeline: preprocessing + logistic regression
logreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", LogisticRegression(max_iter=5000, solver="saga", penalty="l1"))  
    # saga supports both L1 and L2
])

# Parameter distributions for random search
param_distributions = {
    "model__C": loguniform(1e-3, 1e3),   # inverse regularization strength
    "model__penalty": ["l1", "l2"],      # try both penalties
    "model__solver": ["saga"]            # saga works with both l1 and l2
}

# Randomized search
random_search = RandomizedSearchCV(
    logreg_pipe,
    param_distributions=param_distributions,
    n_iter=20,                # number of random samples
    cv=3,
    scoring="balanced_accuracy",
    n_jobs=-1,
    random_state=42
)

# Fit
random_search.fit(X_train, y_train)

print("Best Params:", random_search.best_params_)

# Evaluate on test set
y_pred_test = random_search.predict(X_test)
y_pred_proba = random_search.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_proba)
test_bal_acc = balanced_accuracy_score(y_test, y_pred_test)

print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print(f"Test Balanced Accuracy: {test_bal_acc:.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
mend_results.append({
    "Model": "Logistic Regression (Random Search)",
    "Best Params": random_search.best_params_,
    "Test Accuracy": test_acc,
    "Test AUC": test_auc,
    "Test Balanced Accuracy": test_bal_acc
})

### Log Regession: Grid Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)
# Pipeline: preprocessing + logistic regression
logreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", LogisticRegression(max_iter=5000, solver="saga"))  
    # saga supports both L1 and L2
])

# Grid of hyperparameters
param_grid = {
    "model__C": [0.001, 0.01, 0.1, 1, 10, 100],   # regularization strength
    "model__penalty": ["l1", "l2"]                # L1 (Lasso) or L2 (Ridge)
}

# Grid search
grid = GridSearchCV(
    logreg_pipe,
    param_grid,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1
)

# Fit
grid.fit(X_train, y_train)

# Best parameters
print("Best Params:", grid.best_params_)

# Evaluate on test set
y_pred_test = grid.predict(X_test)
y_pred_proba = grid.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
mend_results.append({
    "Model": "Logistic Regression (Grid Search)",
    "Best Params": grid.best_params_,
    "Test Accuracy": test_acc,
    "Test AUC": test_auc
})


## Week 5 - Support Vector Machines

For Week 5, include concepts such as support vector machines, the kernel trick, and regularization for support vector machines. 

### SVM Basic

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Step 3: Pipeline: preprocessing + SVM (RBF kernel by default)
svm_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", SVC(kernel="rbf", probability=True, random_state=42))
])

# Fit
svm_pipe.fit(X_train, y_train)

# Predictions
y_pred_test = svm_pipe.predict(X_test)
y_pred_proba = svm_pipe.predict_proba(X_test)[:, 1]

# Metrics
print("Basic SVM Accuracy:", accuracy_score(y_test, y_pred_test))
print("Basic SVM AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
mend_results.append({
    "Model": "SVM",
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

### SVM Random Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Pipeline: preprocessing + SVM
svm_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", SVC(probability=True, random_state=42))
])

# Parameter distributions
param_distributions = {
    "model__kernel": ["linear", "rbf"],
    "model__C": loguniform(1e-3, 1e3),   # regularization strength
    "model__gamma": ["scale", "auto", 0.01, 0.1, 1]  # only relevant for RBF
}

# Randomized search
svm_random = RandomizedSearchCV(
    svm_pipe,
    param_distributions=param_distributions,
    n_iter=20,                # number of random samples
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=42
)

# Fit
svm_random.fit(X_train, y_train)

# Best parameters
print("Best Params:", svm_random.best_params_)

# Evaluate on test set
y_pred_test = svm_random.predict(X_test)
y_pred_proba = svm_random.predict_proba(X_test)[:, 1]

print("Random Search SVM Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Search SVM AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

#store results
mend_results.append({
    "Model": "SVM (Random Search)",
    "Best Params": svm_random.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

### SVM Grid Search + Kernel Trick: Linear vs. RBF

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Pipeline: preprocessing + SVM
svm_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", SVC(kernel="rbf", probability=True, random_state=42))
])

# Grid of hyperparameters (RBF only)
param_grid = {
    "model__C": [0.01, 0.1, 1, 10, 100],
    "model__gamma": [0.001, 0.01, 0.1, 1, "scale"]
}

# Grid search
svm_grid = GridSearchCV(
    svm_pipe,
    param_grid,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    return_train_score=True
)

# Fit
svm_grid.fit(X_train, y_train)

# Collect results into DataFrame
cv_results = pd.DataFrame(svm_grid.cv_results_)

# Pivot table for heatmap (mean test AUC)
heatmap_data = cv_results.pivot(
    index="param_model__C",
    columns="param_model__gamma",
    values="mean_test_score"
)

# Plot heatmap
plt.figure(figsize=(8,6))
sns.heatmap(heatmap_data, annot=True, fmt=".3f", cmap="viridis")
plt.title("SVM RBF Kernel: AUC across C and gamma")
plt.ylabel("C (Regularization)")
plt.xlabel("Gamma (Kernel Width)")
plt.show()

# Best params + score
print("Best Params:", svm_grid.best_params_)
print("Best CV AUC:", svm_grid.best_score_)

# Evaluate on test set
y_pred_test = svm_grid.predict(X_test)
y_pred_proba = svm_grid.predict_proba(X_test)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred_test))
print("Test AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

#store results
mend_results.append({
    "Model": "SVM (Grid Search)",
    "Best Params": svm_grid.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

## Week 6 - Decision Trees and Random Forests 

For Week 6, include concepts such as decision trees and random forests.

### Decision Tree Classifier

In [ ]:
# --- Step 1: Reduce dataset to 20% (stratified) ---
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,
    stratify=y_mend_binary_15,
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# --- Step 2: Train/test split (80/20 of reduced set) ---
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# --- Pipeline: preprocessing + Decision Tree with class weights ---
dt_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", DecisionTreeClassifier(
        random_state=42,
        class_weight="balanced"   # <-- handle class imbalance
    ))
])

# --- Parameter distributions for random search ---
dt_param_dist = {
    "model__max_depth": [None, 3, 5, 10, 20],
    "model__min_samples_split": randint(2, 20),
    "model__min_samples_leaf": randint(1, 10),
    "model__criterion": ["gini", "entropy", "log_loss"]
}

# --- Randomized search ---
dt_random = RandomizedSearchCV(
    dt_pipe,
    dt_param_dist,
    n_iter=20,              # number of random draws
    cv=3,
    scoring="roc_auc",
    n_jobs=1,
    random_state=42,
    return_train_score=True
)

# --- Fit ---
with parallel_backend("threading"):
    dt_random.fit(X_train, y_train)


# --- Evaluate ---
y_pred_test = dt_random.predict(X_test)
y_pred_proba = dt_random.predict_proba(X_test)[:, 1]

print("Decision Tree Best Params:", dt_random.best_params_)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_test))
print("Decision Tree AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# --- Store results ---
mend_results.append({
    "Model": "Decision Tree (Random Search, class_weight=balanced)",
    "Best Params": dt_random.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

# ---- Feature Importances ----
best_dt = dt_random.best_estimator_
dt_model = best_dt.named_steps["model"]

# Get feature names from the preprocessor
feature_names = best_dt.named_steps["preprocessor"].get_feature_names_out()

# Map importances back to feature names
importances = pd.Series(dt_model.feature_importances_, index=feature_names)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(
    x=importances_sorted.head(20),
    y=importances_sorted.head(20).index,
    palette="viridis"
)
plt.title("Top 20 Feature Importances - Decision Tree (Random Search)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

### Random Forest Baseline

In [ ]:
# --- Step 1: Reduce dataset to 20% (stratified) ---
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,
    stratify=y_mend_binary_15,
    random_state=42
)

# --- Step 2: Train/test split (80/20 of reduced set) ---
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,
    stratify=y_small,
    random_state=42
)

# --- Pipeline: preprocessing + Random Forest ---
rf_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", RandomForestClassifier(
        n_estimators=200,       # number of trees
        max_depth=None,         # let trees expand fully
        random_state=42,
        class_weight="balanced" # handle class imbalance
    ))
])

# --- Fit ---
with parallel_backend("threading"):
    rf_pipe.fit(X_train, y_train)

# --- Evaluate ---
y_pred_test = rf_pipe.predict(X_test)
y_pred_proba = rf_pipe.predict_proba(X_test)[:, 1]

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Forest AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# --- Feature Importances ---
rf_model = rf_pipe.named_steps["model"]
feature_names = rf_pipe.named_steps["preprocessor"].get_feature_names_out()

importances = pd.Series(rf_model.feature_importances_, index=feature_names)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(
    x=importances_sorted.head(20),
    y=importances_sorted.head(20).index,
    palette="viridis"
)
plt.title("Top 20 Feature Importances - Random Forest (Baseline)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

RocCurveDisplay.from_estimator(rf_pipe, X_test, y_test)
plt.title("ROC Curve - Random Forest (Baseline)")
plt.show()


PrecisionRecallDisplay.from_estimator(rf_pipe, X_test, y_test)
plt.title("Precision-Recall Curve - Random Forest (Baseline)")
plt.show()


ConfusionMatrixDisplay.from_estimator(rf_pipe, X_test, y_test, cmap="Blues")
plt.title("Confusion Matrix - Random Forest (Baseline)")
plt.show()


# Sort by predicted probability
results = pd.DataFrame({"y_true": y_test, "y_score": y_pred_proba})
results = results.sort_values("y_score", ascending=False).reset_index(drop=True)
results["cumulative_positive"] = results["y_true"].cumsum()
results["percent_samples"] = (results.index + 1) / len(results)
results["percent_positives"] = results["cumulative_positive"] / results["y_true"].sum()

plt.figure(figsize=(8,6))
plt.plot(results["percent_samples"], results["percent_positives"], label="Model")
plt.plot([0,1],[0,1], linestyle="--", color="gray", label="Random")
plt.xlabel("Fraction of Samples")
plt.ylabel("Fraction of Positives Captured")
plt.title("Cumulative Gain Curve - Random Forest")
plt.legend()
plt.grid(True)
plt.show()

with parallel_backend("threading"):
    perm_importance = permutation_importance(rf_pipe, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
    
# Align feature names with permutation importance output
fitted_feature_names = rf_pipe.named_steps["preprocessor"].get_feature_names_out()
n_importances = len(perm_importance.importances_mean)

# Slice feature names if there's a mismatch
aligned_feature_names = fitted_feature_names[:n_importances]

perm_sorted = pd.Series(
    perm_importance.importances_mean,
    index=aligned_feature_names
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=perm_sorted.head(20), y=perm_sorted.head(20).index, palette="magma")
plt.title("Top 20 Features - Permutation Importance")
plt.xlabel("Importance (mean decrease in score)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# --- Store results ---
mend_results.append({
    "Model": "Random Forest (Baseline)",
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})


### RF w/ Random Search

In [ ]:
# --- Step 1: Reduce dataset to 20% (stratified) ---
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,
    stratify=y_mend_binary_15,
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# --- Step 2: Train/test split (80/20 of reduced set) ---
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# --- Pipeline: preprocessing + Random Forest with class weights ---
rf_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", RandomForestClassifier(
        random_state=42,
        class_weight="balanced"   # <-- key change
    ))
])

# --- Parameter distributions for random search ---
rf_param_dist = {
    "model__n_estimators": randint(100, 500),
    "model__max_depth": [None, 5, 10, 20],
    "model__min_samples_split": randint(2, 10),
    "model__min_samples_leaf": randint(1, 5)
}

# --- Randomized search ---
rf_random = RandomizedSearchCV(
    rf_pipe,
    rf_param_dist,
    n_iter=20,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

# --- Fit ---
with parallel_backend("threading"):
    rf_random.fit(X_train, y_train)

# --- Evaluate ---
y_pred_test = rf_random.predict(X_test)
y_pred_proba = rf_random.predict_proba(X_test)[:, 1]

print("Random Forest Best Params:", rf_random.best_params_)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Forest AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# --- Store results ---
mend_results.append({
    "Model": "Random Forest (Random Search, class_weight=balanced)",
    "Best Params": rf_random.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})

# ---- Feature Importances ----
best_rf = rf_random.best_estimator_
rf_model = best_rf.named_steps["model"]

# Get feature names from the preprocessor
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

# Map importances back to feature names
importances = pd.Series(rf_model.feature_importances_, index=feature_names)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(
    x=importances_sorted.head(20),
    y=importances_sorted.head(20).index,
    palette="viridis"
)
plt.title("Top 20 Feature Importances - Random Forest (class_weight=balanced)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


# --- ROC Curve ---
RocCurveDisplay.from_estimator(best_rf, X_test, y_test)
plt.title("ROC Curve - Random Forest (Random Search)")
plt.show()

# --- Precision-Recall Curve ---
PrecisionRecallDisplay.from_estimator(best_rf, X_test, y_test)
plt.title("Precision-Recall Curve - Random Forest (Random Search)")
plt.show()

# --- Confusion Matrix Heatmap ---
ConfusionMatrixDisplay.from_estimator(best_rf, X_test, y_test, cmap="Blues")
plt.title("Confusion Matrix - Random Forest (Random Search)")
plt.show()

# --- Permutation Importance (more robust than Gini) ---
with parallel_backend("threading"):
    perm_importance = permutation_importance(
        best_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )
    
# Align feature names with the actual number of features used
n_importances = len(perm_importance.importances_mean)
aligned_feature_names = feature_names[:n_importances]

perm_sorted = pd.Series(
    perm_importance.importances_mean,
    index=aligned_feature_names
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=perm_sorted.head(20), y=perm_sorted.head(20).index, palette="magma")
plt.title("Top 20 Features - Permutation Importance")
plt.xlabel("Importance (mean decrease in score)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
# Align feature names with the actual number of features used
n_importances = len(perm_importance.importances_mean)
aligned_feature_names = feature_names[:n_importances]

perm_sorted = pd.Series(
    perm_importance.importances_mean,
    index=aligned_feature_names
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=perm_sorted.head(20), y=perm_sorted.head(20).index, palette="magma")
plt.title("Top 20 Features - Permutation Importance")
plt.xlabel("Importance (mean decrease in score)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

### RF w/ Grid Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_mend_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_mend.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Pipeline: preprocessing + Random Forest
rf_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", RandomForestClassifier(random_state=42, class_weight="balanced"))
])

# Grid of hyperparameters
rf_param_grid = {
    "model__n_estimators": [50, 100],
    "model__max_depth": [5, 10, 25],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2]
}

# Grid search
rf_grid = GridSearchCV(
    rf_pipe,
    rf_param_grid,
    cv=2,
    scoring="roc_auc",
    n_jobs=-1
)

with parallel_backend("threading"):
    rf_grid.fit(X_train, y_train)
    
# Evaluate
y_pred_test = rf_grid.predict(X_test)
y_pred_proba = rf_grid.predict_proba(X_test)[:, 1]

print("Random Forest Best Params:", rf_grid.best_params_)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Forest AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
mend_results.append({
    "Model": "Random Forest (Grid Search, class_weight=balanced)",
    "Best Params": rf_grid.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})

importances = rf_grid.best_estimator_.named_steps["model"].feature_importances_
feature_names = rf_grid.best_estimator_.named_steps["preprocessor"].get_feature_names_out()
pd.Series(importances, index=feature_names).sort_values(ascending=False).head(20)

# Get the best fitted pipeline
best_rf = rf_grid.best_estimator_

# Extract the trained RandomForest model
rf_model = best_rf.named_steps["model"]

# Get feature names from the preprocessor
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

# Pair feature names with importances
importances = pd.Series(rf_model.feature_importances_, index=feature_names)

# Sort by importance
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(x=importances_sorted.head(20), y=importances_sorted.head(20).index, palette="viridis")
plt.title("Top 20 Feature Importances - Random Forest")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


# --- ROC Curve ---
RocCurveDisplay.from_estimator(best_rf, X_test, y_test)
plt.title("ROC Curve - Random Forest (Grid Search)")
plt.show()

# --- Precision-Recall Curve ---
PrecisionRecallDisplay.from_estimator(best_rf, X_test, y_test)
plt.title("Precision-Recall Curve - Random Forest (Grid Search)")
plt.show()

# --- Confusion Matrix Heatmap ---
ConfusionMatrixDisplay.from_estimator(best_rf, X_test, y_test, cmap="Blues")
plt.title("Confusion Matrix - Random Forest (Grid Search)")
plt.show()

# Transform X_test with the preprocessor to get the actual feature matrix
X_test_transformed = best_rf.named_steps["preprocessor"].transform(X_test)

# Get the feature names aligned with the transformed matrix
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

print("X_test_transformed shape:", X_test_transformed.shape)
print("Number of feature names:", len(feature_names))

with parallel_backend("threading"):
    perm_importance = permutation_importance(
        best_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
)

# Align lengths
perm_sorted = pd.Series(
    perm_importance.importances_mean,
    index=feature_names[:len(perm_importance.importances_mean)]
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=perm_sorted.head(20), y=perm_sorted.head(20).index, palette="magma")
plt.title("Top 20 Features - Permutation Importance")
plt.xlabel("Importance (mean decrease in score)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


In [ ]:
# Transform X_test with the preprocessor to get the actual feature matrix
X_test_transformed = best_rf.named_steps["preprocessor"].transform(X_test)

# Get the feature names aligned with the transformed matrix
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

print("X_test_transformed shape:", X_test_transformed.shape)
print("Number of feature names:", len(feature_names))
with parallel_backend("threading"):
    perm_importance = permutation_importance(
        best_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )

# Align lengths
perm_sorted = pd.Series(
    perm_importance.importances_mean,
    index=feature_names[:len(perm_importance.importances_mean)]
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=perm_sorted.head(20), y=perm_sorted.head(20).index, palette="magma")
plt.title("Top 20 Features - Permutation Importance")
plt.xlabel("Importance (mean decrease in score)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

## Week 7 - Milestone 1

### Review Results

In [ ]:
# Convert list of dicts into DataFrame
mend_results_df = pd.DataFrame(mend_results)

# Sort by test_r2 (descending) or test_rmse (ascending)
# results_df_sorted = results_df.sort_values(by="test_r2", ascending=False)

print(mend_results_df)

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=mend_results_df, x="model", y="test_r2", palette="viridis")
plt.title("Model Comparison (Test R²)")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8,5))
sns.barplot(data=mend_results_df, x="model", y="test_rmse", palette="magma")
plt.title("Model Comparison (Test RMSE)")
plt.xticks(rotation=45)
plt.show()

# USDOT Carrier On-Time Dataset

## Data Prep

### Data Prep Functions

In [ ]:
def engineer_usdot_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # --- Datetime features ---
    df['fl_date'] = pd.to_datetime(
        df['fl_date'].astype(str),   # convert category → string
        format="%m/%d/%Y %I:%M:%S %p",  # matches "6/27/2025 12:00:00 AM"
        errors="coerce"
    )
    df['year'] = df['fl_date'].dt.year
    df['month'] = df['fl_date'].dt.month
    df['day'] = df['fl_date'].dt.day
    df['day_of_week'] = df['fl_date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)

    # Scheduled departure hour
    df['crs_dep_hour'] = (df['crs_dep_time'] // 100).astype(int)
    df['part_of_day'] = pd.cut(
        df['crs_dep_hour'],
        bins=[-1,5,11,17,21,24],
        labels=['night','morning','afternoon','evening','late_night']
    )
    
    # --- Route features ---
    df['route'] = df['origin'].astype(str) + "_" + df['dest'].astype(str)
    
    major_hubs = {'ATL','ORD','DFW','DEN','JFK','LAX','SFO','CLT','LAS','PHX','MIA'}
    df['is_hub_origin'] = df['origin'].isin(major_hubs).astype(int)
    df['is_hub_dest'] = df['dest'].isin(major_hubs).astype(int)
    
    # --- Operational features ---
    df['taxi_time'] = df[['taxi_out','taxi_in']].sum(axis=1, skipna=True)
    df['air_time_ratio'] = np.where(
        df['crs_elapsed_time']>0,
        df['air_time'] / df['crs_elapsed_time'],
        np.nan
    )
    
    # Flight distance buckets (custom)
    df['distance_bucket'] = pd.cut(
        df['distance'],
        bins=[0,500,1000,1500,2000,3000,10000],
        labels=['short','medium','long','xlong','xxlong','ultra']
    )
    
    # --- Delay flags ---
    df['is_delayed_15'] = (df['dep_delay'] >= 15).astype(int)
    df['is_delayed_60'] = (df['dep_delay'] >= 60).astype(int)
    
    # --- Fill missing values ---
    if 'tail_num' in df.columns:
        if pd.api.types.is_categorical_dtype(df['tail_num']):
            df['tail_num'] = df['tail_num'].cat.add_categories(["UNKNOWN_TAIL"])
        df['tail_num'] = df['tail_num'].fillna("UNKNOWN_TAIL")

    if 'cancellation_code' in df.columns:
        if pd.api.types.is_categorical_dtype(df['cancellation_code']):
            df['cancellation_code'] = df['cancellation_code'].cat.add_categories(["NONE"])
        df['cancellation_code'] = df['cancellation_code'].fillna("NONE")
    
    return df

### Data Prep Steps

In [ ]:
# Use glob to find all matching CSV files
all_files = glob.glob(os.path.join(data_path, "T_ONTIME_REPORTING_2025*.csv"))

# Read and combine them
dfs = [pd.read_csv(f) for f in all_files]
combined_df = pd.concat(dfs, ignore_index=True)

print("Files combined:", len(all_files))
print("Final shape:", combined_df.shape)

# Drop diverted columns
combined_df = combined_df.drop(combined_df.filter(regex=r"^DIV\d+").columns, axis=1)

df_usdot = optimize_dataframe(
    combined_df,
    datetime_cols=['fl_date'],
    fillna=True
)
df_usdot = clean_column_names(df_usdot)

# Remove outliers from dataset
df_usdot_clean = df_usdot[df_usdot['dep_delay'] >= -30].copy()
print("Cleaned shape:", df_usdot_clean.shape)

In [ ]:
file_name = "FAA_AC_REGISTRATION_2021.csv"
df_reg = pd.read_csv(data_path + file_name)

# Left join on stripped tail_num
df_usdot_clean_reg = df_usdot_clean.merge(
    df_reg,
    how="left",
    left_on=df_usdot_clean["tail_num"].str.lstrip("N"),
    right_on="N-NUMBER"
)

# Impute missing values with mode per column
mode_dict = {col: df_reg[col].mode()[0] for col in df_reg.columns if col != "N-NUMBER"}
df_usdot_clean_reg = df_usdot_clean_reg.fillna(mode_dict)

# Drop duplicate join key
df_usdot_clean_reg = df_usdot_clean_reg.drop(columns=["N-NUMBER"])

In [ ]:
# Engineer features
df_usdot_clean_eng = engineer_usdot_features(df_usdot_clean_reg)

# Get column categories
df_usdot_id_cols = ['op_carrier_airline_id', 'origin_airport_id', 'origin_airport_seq_id', 'origin_city_market_id', 'origin_state_fips', 'origin_wac', 'dest_airport_id', 'dest_airport_seq_id', 'dest_city_market_id', 'dest_state_fips', 'dest_wac', 'crs_dep_time', 'crs_arr_time']
df_usdot_cat_cols = ['op_unique_carrier', 'op_carrier', 'tail_num', 'origin', 'origin_city_name', 'origin_state_abr', 'origin_state_nm', 'dest', 'dest_city_name', 'dest_state_abr', 'dest_state_nm', 'dest_state_fips', 'dest_wac', 'dep_time_blk', 'arr_time_blk', 'cancellation_code,']
df_usdot_date_cols = ['fl_date', ]
df_usdot_target_cols = ['dep_delay', 'dep_delay_new', 'dep_del15', 'dep_delay_group', 'arr_delay', 'arr_delay_new', 'arr_del15', 
                        'arr_delay_group', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
                        'is_delayed_15', 'is_delayed_60']
df_usdot_feature_cols = [col for col in df_usdot_clean_eng.columns if col not in df_usdot_id_cols + df_usdot_cat_cols + df_usdot_date_cols + df_usdot_target_cols]

# drop leakage columns
TARGET_COLUMN = 'dep_delay'
df_usdot_leakage_cols = [x for x in df_usdot_target_cols if x != TARGET_COLUMN]
df_usdot_clean_filter = df_usdot_clean_eng.drop(df_usdot_leakage_cols + df_usdot_id_cols + df_usdot_date_cols, axis=1, errors="ignore").copy()
# Export df to csv
df_usdot_clean_filter.to_csv(data_path + 'USDOTDelayData_Cleaned.csv', index=False)

reg_prep_usdot, tree_prep_usdot, X_usdot, y_usdot_numeric = build_dual_preprocessors(df_usdot_clean_filter, 
                                                                                 target=TARGET_COLUMN, 
                                                                                 feature_cols=df_usdot_feature_cols, 
                                                                                 high_card_threshold=30, scale_numeric=True)

# Create binary target for classification (15 min delay threshold)
y_usdot_binary_15 = (y_usdot_numeric >= 15).astype(int)

# Ensure categorical columns are string type (Some reg data has mixed types)
cat_cols = ['cancellation_code', 'MFR', 'TYPE-ACFT', 'AC-WEIGHT', 'part_of_day', 'distance_bucket']
for col in cat_cols:
    X_usdot[col] = X_usdot[col].astype(str)

# Create seperate transformed datasets for regression and tree models
X_reg_usdot = transform_with_names(reg_prep_usdot, X_usdot, y_usdot_numeric)
X_tree_usdot = transform_with_names(tree_prep_usdot, X_usdot, y_usdot_numeric)

## Week 1: Linear Regression 1

### Week 1 Helper Functions

In [ ]:
def regression_summary(X, y):
    """
    Fits an OLS regression model using statsmodels and prints the summary.
    """
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    return model.summary()

def fit_polynomial_regression(X, y, degree=2):
    """
    Fits a polynomial regression model and returns the fitted model and transformed features.
    """
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, y)
    return model, poly


def calculate_vif(df, features=None, vif_thresh=10.0):
    """
    Calculate Variance Inflation Factor (VIF) safely:
    - Removes constant columns
    - Removes perfectly collinear columns
    - Returns sorted VIF table
    
    Parameters:
        df (pd.DataFrame): DataFrame with numeric features
        features (list): Optional list of features to check; defaults to all numeric
        vif_thresh (float): Threshold for flagging high VIF
    
    Returns:
        pd.DataFrame: VIF table
    """
    # Select numeric columns if features not provided
    if features is None:
        features = df.select_dtypes(include=[np.number]).columns.tolist()
    
    X = df[features].copy()
    
    # 1. Drop constant columns
    constant_cols = [col for col in X.columns if X[col].nunique() <= 1]
    if constant_cols:
        print(f"Dropping constant columns: {constant_cols}")
        X.drop(columns=constant_cols, inplace=True)
    
    # 2. Drop perfectly collinear columns
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    perfect_corr_cols = [col for col in upper.columns if any(upper[col] == 1.0)]
    if perfect_corr_cols:
        print(f"Dropping perfectly collinear columns: {perfect_corr_cols}")
        X.drop(columns=perfect_corr_cols, inplace=True)
    
    # 3. Calculate VIF
    X_const = X.assign(const=1)
    vif_data = pd.DataFrame({
        "feature": X.columns,
        "VIF": [variance_inflation_factor(X_const.values, i) for i in range(len(X.columns))]
    })
    
    # 4. Sort by VIF
    vif_data.sort_values(by="VIF", ascending=False, inplace=True)
    
    # 5. Flag high VIF
    vif_data["High_VIF"] = vif_data["VIF"] > vif_thresh
    
    return vif_data

### Linear Regression

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_usdot, y_usdot_numeric, test_size=0.2, random_state=42
)

# Build pipeline
linreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", LinearRegression())
])

# Fit
linreg_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = linreg_pipe.predict(X_train)
y_pred_test = linreg_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

# Store results
usdot_results.append({
    'model': 'Linear Regression',
    'train_r2': train_r2,
    'test_r2': test_r2, 
    'test_rmse': test_rmse
})

### Polynomial Regression

Reduced the number of features

In [ ]:
# Choose 5 features explicitly
selected_features = ["air_time", "distance", "crs_dep_hour", "is_hub_dest", "is_weekend",
                     "crs_elapsed_time", "taxi_time", "air_time_ratio"]

# Reduce dataset to 20% of original, but only keep those 5 columns
X_small, _, y_small, _ = train_test_split(
    X_usdot[selected_features],  # <-- subset here
    y_usdot_numeric,
    train_size=0.2,
    random_state=42
)


# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)


# Define a new preprocessor for just these features
reg_prep_small = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), selected_features)
    ]
)

poly_reg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_small),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("model", LinearRegression())
])

# Fit
poly_reg_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = poly_reg_pipe.predict(X_train)
y_pred_test = poly_reg_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

# Store results
usdot_results.append({
    'model': 'Polynomial Regression',
    'train_r2': train_r2,
    'test_r2': test_r2,
    'test_rmse': test_rmse
})

### VIF: Variable Inflation Factor

In [ ]:
X_reg_usdot_sample = X_reg_usdot.sample(n=50000, random_state=42) if len(X_reg_usdot) > 10000 else X_reg_usdot
vif_table = calculate_vif(X_reg_usdot_sample, features=X_reg_usdot_sample.columns.tolist(), vif_thresh=10.0)
print(vif_table)

## Week 2: Linear Regression 2

### Lasso Regression

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Build pipeline with preprocessing + Lasso
lasso_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", Lasso(alpha=0.1, max_iter=10000, random_state=42))
])

# Fit
lasso_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = lasso_pipe.predict(X_train)
y_pred_test = lasso_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

#store results
usdot_results.append({
    'model': 'Lasso Regression',
    'train_r2': train_r2,
    'test_r2': test_r2,
    'test_rmse': test_rmse
})  

### Lasso Grid Search

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Pipeline: preprocessing + Lasso
lasso_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", Lasso(max_iter=10000, random_state=42))
])

# Grid of hyperparameters to search
param_grid = {
    "model__alpha": [0.001, 0.01, 0.1, 1, 10]
}

# Grid search with 5-fold CV
grid = GridSearchCV(
    lasso_pipe,
    param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit grid search
grid.fit(X_train, y_train)

# Best parameters
print("Best alpha:", grid.best_params_["model__alpha"])

# Evaluate on test set
y_pred_test = grid.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

# Store results
usdot_results.append({
    'model': 'Lasso Regression (Tuned)',
    'best_alpha': grid.best_params_["model__alpha"],
    'train_r2': grid.best_score_,
    'test_r2': test_r2,
    'test_rmse': test_rmse
})

### Ridge Regression

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Build pipeline with preprocessing + Ridge
ridge_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", Ridge(alpha=1.0, max_iter=10000, random_state=42))
])

# Fit
ridge_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = ridge_pipe.predict(X_train)
y_pred_test = ridge_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

#store results
usdot_results.append({
    'model': 'Ridge Regression',
    'train_r2': train_r2,
    'test_r2': test_r2, 
    'test_rmse': test_rmse
})

### Ridge Regression Grid Search

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)
# Pipeline: preprocessing + Ridge
ridge_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", Ridge(max_iter=10000, random_state=42))
])

# Grid of hyperparameters
ridge_param_grid = {
    "model__alpha": [0.01, 0.1, 1, 10, 100]
}

ridge_grid = GridSearchCV(
    ridge_pipe,
    ridge_param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

ridge_grid.fit(X_train, y_train)

print("Best Ridge alpha:", ridge_grid.best_params_["model__alpha"])

y_pred_test = ridge_grid.predict(X_test)
print("Ridge Test R²:", r2_score(y_test, y_pred_test))
print("Ridge Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))

#store results
usdot_results.append({
    'model': 'Ridge Regression (Tuned)',
    'best_alpha': ridge_grid.best_params_["model__alpha"],
    'train_r2': ridge_grid.best_score_,
    'test_r2': r2_score(y_test, y_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test))
})

### Elastic Net

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# Build pipeline with preprocessing + Elastic Net
elasticnet_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000, random_state=42))
])
# Fit
elasticnet_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = elasticnet_pipe.predict(X_train)
y_pred_test = elasticnet_pipe.predict(X_test)

# Metrics
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
print(f"Test RMSE: {test_rmse:.3f}")

#store results
usdot_results.append({
    'model': 'Elastic Net Regression',
    'train_r2': train_r2,
    'test_r2': test_r2, 
    'test_rmse': test_rmse
})

### Elastic Net Grid Search

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)
# Pipeline: preprocessing + ElasticNet
elastic_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", ElasticNet(max_iter=10000, random_state=42))
])

# Grid of hyperparameters
elastic_param_grid = {
    "model__alpha": [0.001, 0.01, 1, 10],
    "model__l1_ratio": [0.2,  0.8]  # balance between L1 and L2
}

elastic_grid = GridSearchCV(
    elastic_pipe,
    elastic_param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

elastic_grid.fit(X_train, y_train)

print("Best ElasticNet params:", elastic_grid.best_params_)

y_pred_test = elastic_grid.predict(X_test)
print("ElasticNet Test R²:", r2_score(y_test, y_pred_test))
print("ElasticNet Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))

#store results
usdot_results.append({
    'model': 'Elastic Net Regression (Tuned)',
    'best_params': elastic_grid.best_params_,
    'train_r2': elastic_grid.best_score_,
    'test_r2': r2_score(y_test, y_pred_test),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test))
})

## Week 3: Linear Regression 3

### Forward & Backward Selection: Linear Regression

In [ ]:
# --- Shrink dataset to 20% ---
X_sample, _, y_sample, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    test_size=0.8,
    random_state=42
)

# Preprocess once
X_proc = reg_prep_usdot.fit_transform(X_sample, y_sample)
y = y_sample

# --- Forward Selection (limit to 30 features, step=2) ---
forward_rmse = []
k_range_fwd = range(1, min(31, X_proc.shape[1] + 1), 2)

for k in k_range_fwd:
    sfs = SequentialFeatureSelector(
        estimator=LinearRegression(),
        n_features_to_select=k,
        direction="forward",
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    sfs.fit(X_proc, y)
    mask = sfs.get_support()
    # Evaluate using CV on the reduced feature set
    scores = cross_val_score(
        LinearRegression(),
        X_proc[:, mask],
        y,
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    rmse = np.sqrt(-scores.mean())
    forward_rmse.append(rmse)

best_idx_fwd = np.argmin(forward_rmse)
best_k_fwd = list(k_range_fwd)[best_idx_fwd]
best_rmse_fwd = forward_rmse[best_idx_fwd]

usdot_results.append({
    'model': 'Forward Selection Linear Regression',
    'num_features': best_k_fwd,
    'best_cv_rmse': best_rmse_fwd,
    'rmse_curve': forward_rmse,
    'k_range': list(k_range_fwd)
})

# --- Backward Selection (step size=3, early stopping) ---
backward_rmse = []
k_range_bwd = range(1, X_proc.shape[1] + 1, 3)

tolerance = 1e-3
best_rmse_so_far = np.inf

for k in k_range_bwd:
    sbs = SequentialFeatureSelector(
        estimator=LinearRegression(),
        n_features_to_select=k,
        direction="backward",
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    sbs.fit(X_proc, y)
    mask = sbs.get_support()

    # Evaluate this subset with CV
    scores = cross_val_score(
        LinearRegression(),
        X_proc[:, mask],
        y,
        scoring="neg_mean_squared_error",
        cv=2,
        n_jobs=-1
    )
    rmse = np.sqrt(-scores.mean())
    backward_rmse.append(rmse)

    # Early stopping if curve flattens
    if best_rmse_so_far - rmse < tolerance:
        print(f"Stopping early at {k} features (no significant improvement).")
        break
    best_rmse_so_far = min(best_rmse_so_far, rmse)


best_idx_bwd = np.argmin(backward_rmse)
best_k_bwd = list(k_range_bwd)[:len(backward_rmse)][best_idx_bwd]
best_rmse_bwd = backward_rmse[best_idx_bwd]

usdot_results.append({
    'model': 'Backward Selection Linear Regression',
    'num_features': best_k_bwd,
    'best_cv_rmse': best_rmse_bwd,
    'rmse_curve': backward_rmse,
    'k_range': list(k_range_bwd)[:len(backward_rmse)]
})

# --- Plot curves ---
plt.figure(figsize=(8,5))
plt.plot(k_range_fwd, forward_rmse, marker="o", label="Forward Selection")
plt.plot(list(k_range_bwd)[:len(backward_rmse)], backward_rmse, marker="s", label="Backward Selection")
plt.xlabel("Number of Features Selected")
plt.ylabel("CV RMSE")
plt.title("Forward vs Backward Selection (optimized)")
plt.legend()
plt.grid(True)
plt.show()

### PCR
Principal Component Regression

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# PCR pipeline: preprocessing → PCA → Linear Regression
pcr_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),   # your ColumnTransformer
    ("pca", PCA()),                    # dimensionality reduction
    ("model", LinearRegression())
])

# Grid search over number of components
max_components = min(X_train.shape[0], X_train.shape[1])

param_grid = {
    "pca__n_components": list(range(5, max_components+1, 5))
}

pcr_grid = GridSearchCV(
    pcr_pipe,
    param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit
pcr_grid.fit(X_train, y_train)

# Best number of components
print("Best n_components:", pcr_grid.best_params_["pca__n_components"])

# Evaluate on test set
y_pred_test = pcr_grid.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"PCR Test R²: {test_r2:.3f}")
print(f"PCR Test RMSE: {test_rmse:.3f}")

# Store results for comparison
usdot_results.append({
    "Model": "PCR",
    "Best Params": pcr_grid.best_params_,
    "Test R²": test_r2,
    "Test RMSE": test_rmse
})


### PLSR
Partial Least Squares Regression

In [ ]:
# Reduce dataset to 20% of original
X_small, _, y_small, _ = train_test_split(
    X_usdot, y_usdot_numeric,
    train_size=0.2,
    stratify=None,   # or stratify=y_usdot_binary_15 if classification
    random_state=42
)

# Train/test split on reduced set
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small,
    test_size=0.2,
    random_state=42
)

# PLSR pipeline: preprocessing → PLSRegression
pls_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),   # your ColumnTransformer
    ("model", PLSRegression())
])

# Grid search over number of components
param_grid = {
    "model__n_components": [2, 5, 10, 20, 40]  # tune based on dataset size
}

pls_grid = GridSearchCV(
    pls_pipe,
    param_grid,
    cv=3,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Fit
pls_grid.fit(X_train, y_train)

# Best number of components
print("Best n_components:", pls_grid.best_params_["model__n_components"])

# Evaluate on test set
y_pred_test = pls_grid.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"PLSR Test R²: {test_r2:.3f}")
print(f"PLSR Test RMSE: {test_rmse:.3f}")

# Store results for comparison
usdot_results.append({
    "Model": "PLSR",
    "Best Params": pls_grid.best_params_,
    "Test R²": test_r2,
    "Test RMSE": test_rmse
})

## Week 4: Log Regression and Feature Selection

### Log Regression: Basic

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)


# Build pipeline: preprocessing + logistic regression
logreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", LogisticRegression(max_iter=1000, solver="liblinear"))
])

# Fit
logreg_pipe.fit(X_train, y_train)

# Predictions
y_pred_train = logreg_pipe.predict(X_train)
y_pred_test = logreg_pipe.predict(X_test)
y_pred_proba = logreg_pipe.predict_proba(X_test)[:, 1]

# Metrics
train_acc = accuracy_score(y_train, y_pred_train)
test_acc = accuracy_score(y_test, y_pred_test)
train_bal_acc = balanced_accuracy_score(y_train, y_pred_train)
test_bal_acc = balanced_accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Train Accuracy: {train_acc:.3f}")
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Train Balanced Accuracy: {train_bal_acc:.3f}")
print(f"Test Balanced Accuracy: {test_bal_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
usdot_results.append({
    "Model": "Logistic Regression",
    "Train Accuracy": train_acc,
    "Test Accuracy": test_acc,
    "Train Balanced Accuracy": train_bal_acc,
    "Test Balanced Accuracy": test_bal_acc,
    "Test AUC": test_auc
})

### Log Regression: Forward & Backward Selection: Light

In [ ]:
# --- Shrink dataset to 20% with stratification on binary target ---
X_sample, _, y_sample, _ = train_test_split(
    X_usdot, y_usdot_binary_15,
    test_size=0.8,
    random_state=42,
    stratify=y_usdot_binary_15
)

# Preprocess once
X_proc = reg_prep_usdot.fit_transform(X_sample, y_sample)
y = y_sample

# Define estimator
logreg = LogisticRegression(max_iter=1000, solver="liblinear")

# --- Forward Selection (limit to 30 features) ---
forward_scores = []
k_range_fwd = range(1, min(31, X_proc.shape[1] + 1))

tolerance = 1e-3
best_score_so_far = -np.inf

for k in k_range_fwd:
    sfs = SequentialFeatureSelector(
        estimator=logreg,
        n_features_to_select=k,
        direction="forward",
        scoring="balanced_accuracy",
        cv=3,
        n_jobs=-1
    )
    sfs.fit(X_proc, y)
    mask = sfs.get_support()
    score = cross_val_score(
        logreg,
        X_proc[:, mask],
        y,
        scoring="balanced_accuracy",
        cv=3
    ).mean()
    forward_scores.append(score)

    # Early stopping if improvement is too small
    if score - best_score_so_far < tolerance:
        print(f"Forward selection stopping early at {k} features (Δ<{tolerance}).")
        break
    best_score_so_far = max(best_score_so_far, score)

# Best forward
best_idx_fwd = np.argmax(forward_scores)
best_k_fwd = list(k_range_fwd)[best_idx_fwd]
best_score_fwd = forward_scores[best_idx_fwd]

usdot_results.append({
    'model': 'Forward Selection Logistic Regression',
    'num_features': best_k_fwd,
    'best_cv_bal_acc': best_score_fwd,
    'score_curve': forward_scores,
    'k_range': list(k_range_fwd)
})

# --- Backward Selection (step size = 3, early stopping) ---
backward_scores = []
n_features = X_proc.shape[1]

tolerance = 1e-3
best_score_so_far = -np.inf

for k in range(n_features, 0, -1):  # from all features down to 1
    sbs = SequentialFeatureSelector(
        estimator=logreg,
        n_features_to_select=k,
        direction="backward",
        scoring="balanced_accuracy",
        cv=3,
        n_jobs=-1
    )
    sbs.fit(X_proc, y)
    mask = sbs.get_support()
    score = cross_val_score(
        logreg,
        X_proc[:, mask],
        y,
        scoring="balanced_accuracy",
        cv=3
    ).mean()
    backward_scores.append(score)

    # Early stopping if curve flattens
    if score - best_score_so_far < tolerance:
        print(f"Backward selection stopping early at {k} features (Δ<{tolerance}).")
        break
    best_score_so_far = max(best_score_so_far, score)

# Reverse scores so they align with decreasing k
backward_scores = backward_scores[::-1]
k_range_bwd = list(range(n_features, n_features - len(backward_scores), -1))[::-1]

# Best backward
best_idx_bwd = np.argmax(backward_scores)
best_k_bwd = list(k_range_bwd)[:len(backward_scores)][best_idx_bwd]
best_score_bwd = backward_scores[best_idx_bwd]

usdot_results.append({
    'model': 'Backward Selection Logistic Regression',
    'num_features': best_k_bwd,
    'best_cv_bal_acc': best_score_bwd,
    'score_curve': backward_scores,
    'k_range': list(k_range_bwd)[:len(backward_scores)]
})

# --- Plot curves ---
plt.figure(figsize=(8,5))
plt.plot(k_range_fwd, forward_scores, marker="o", label="Forward Selection")
plt.plot(list(k_range_bwd)[:len(backward_scores)], backward_scores, marker="s", label="Backward Selection")
plt.xlabel("Number of Features Selected")
plt.ylabel("CV Balanced Accuracy")
plt.title("Forward vs Backward Selection (Logistic Regression)")
plt.legend()
plt.grid(True)
plt.show()

### Log Regression: Random Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

# Pipeline: preprocessing + logistic regression
logreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", LogisticRegression(max_iter=5000, solver="saga", penalty="l1"))  
    # saga supports both L1 and L2
])

# Parameter distributions for random search
param_distributions = {
    "model__C": loguniform(1e-3, 1e3),   # inverse regularization strength
    "model__penalty": ["l1", "l2"],      # try both penalties
    "model__solver": ["saga"]            # saga works with both l1 and l2
}

# Randomized search
random_search = RandomizedSearchCV(
    logreg_pipe,
    param_distributions=param_distributions,
    n_iter=20,                # number of random samples
    cv=3,
    scoring="balanced_accuracy",
    n_jobs=-1,
    random_state=42
)

# Fit
random_search.fit(X_train, y_train)

print("Best Params:", random_search.best_params_)

# Evaluate on test set
y_pred_test = random_search.predict(X_test)
y_pred_proba = random_search.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_proba)
test_bal_acc = balanced_accuracy_score(y_test, y_pred_test)

print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print(f"Test Balanced Accuracy: {test_bal_acc:.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
usdot_results.append({
    "Model": "Logistic Regression (Random Search)",
    "Best Params": random_search.best_params_,
    "Test Accuracy": test_acc,
    "Test AUC": test_auc,
    "Test Balanced Accuracy": test_bal_acc
})

### Log Regession: Grid Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)
# Pipeline: preprocessing + logistic regression
logreg_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", LogisticRegression(max_iter=5000, solver="saga"))  
    # saga supports both L1 and L2
])

# Grid of hyperparameters
param_grid = {
    "model__C": [0.001, 0.01, 0.1, 1, 10, 100],   # regularization strength
    "model__penalty": ["l1", "l2"]                # L1 (Lasso) or L2 (Ridge)
}

# Grid search
grid = GridSearchCV(
    logreg_pipe,
    param_grid,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1
)

# Fit
grid.fit(X_train, y_train)

# Best parameters
print("Best Params:", grid.best_params_)

# Evaluate on test set
y_pred_test = grid.predict(X_test)
y_pred_proba = grid.predict_proba(X_test)[:, 1]

test_acc = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test AUC: {test_auc:.3f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
usdot_results.append({
    "Model": "Logistic Regression (Grid Search)",
    "Best Params": grid.best_params_,
    "Test Accuracy": test_acc,
    "Test AUC": test_auc
})


## Week 5 - Support Vector Machines

For Week 5, include concepts such as support vector machines, the kernel trick, and regularization for support vector machines. 

### SVM Basic

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Step 3: Pipeline: preprocessing + SVM (RBF kernel by default)
svm_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", SVC(kernel="rbf", probability=True, random_state=42))
])

# Fit
svm_pipe.fit(X_train, y_train)

# Predictions
y_pred_test = svm_pipe.predict(X_test)
y_pred_proba = svm_pipe.predict_proba(X_test)[:, 1]

# Metrics
print("Basic SVM Accuracy:", accuracy_score(y_test, y_pred_test))
print("Basic SVM AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
usdot_results.append({
    "Model": "SVM",
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

### SVM Random Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Pipeline: preprocessing + SVM
svm_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", SVC(probability=True, random_state=42))
])

# Parameter distributions
param_distributions = {
    "model__kernel": ["linear", "rbf"],
    "model__C": loguniform(1e-3, 1e3),   # regularization strength
    "model__gamma": ["scale", "auto", 0.01, 0.1, 1]  # only relevant for RBF
}

# Randomized search
svm_random = RandomizedSearchCV(
    svm_pipe,
    param_distributions=param_distributions,
    n_iter=20,                # number of random samples
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=42
)

# Fit
svm_random.fit(X_train, y_train)

# Best parameters
print("Best Params:", svm_random.best_params_)

# Evaluate on test set
y_pred_test = svm_random.predict(X_test)
y_pred_proba = svm_random.predict_proba(X_test)[:, 1]

print("Random Search SVM Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Search SVM AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

#store results
usdot_results.append({
    "Model": "SVM (Random Search)",
    "Best Params": svm_random.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

### SVM Grid Search + Kernel Trick: Linear vs. RBF

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Pipeline: preprocessing + SVM
svm_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", SVC(kernel="rbf", probability=True, random_state=42))
])

# Grid of hyperparameters (RBF only)
param_grid = {
    "model__C": [0.01, 0.1, 1, 10, 100],
    "model__gamma": [0.001, 0.01, 0.1, 1, "scale"]
}

# Grid search
svm_grid = GridSearchCV(
    svm_pipe,
    param_grid,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    return_train_score=True
)

# Fit
svm_grid.fit(X_train, y_train)

# Collect results into DataFrame
cv_results = pd.DataFrame(svm_grid.cv_results_)

# Pivot table for heatmap (mean test AUC)
heatmap_data = cv_results.pivot(
    index="param_model__C",
    columns="param_model__gamma",
    values="mean_test_score"
)

# Plot heatmap
plt.figure(figsize=(8,6))
sns.heatmap(heatmap_data, annot=True, fmt=".3f", cmap="viridis")
plt.title("SVM RBF Kernel: AUC across C and gamma")
plt.ylabel("C (Regularization)")
plt.xlabel("Gamma (Kernel Width)")
plt.show()

# Best params + score
print("Best Params:", svm_grid.best_params_)
print("Best CV AUC:", svm_grid.best_score_)

# Evaluate on test set
y_pred_test = svm_grid.predict(X_test)
y_pred_proba = svm_grid.predict_proba(X_test)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred_test))
print("Test AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

#store results
usdot_results.append({
    "Model": "SVM (Grid Search)",
    "Best Params": svm_grid.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba)
})

## Week 6 - Decision Trees and Random Forests 

For Week 6, include concepts such as decision trees and random forests.

### Decision Tree Classifier

In [ ]:
# --- Step 1: Reduce dataset to 20% (stratified) ---
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,
    stratify=y_usdot_binary_15,
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# --- Step 2: Train/test split (80/20 of reduced set) ---
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# --- Pipeline: preprocessing + Decision Tree with class weights ---
dt_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", DecisionTreeClassifier(
        random_state=42,
        class_weight="balanced"   # <-- handle class imbalance
    ))
])

# --- Parameter distributions for random search ---
dt_param_dist = {
    "model__max_depth": [None, 3, 5, 10, 20],
    "model__min_samples_split": randint(2, 20),
    "model__min_samples_leaf": randint(1, 10),
    "model__criterion": ["gini", "entropy", "log_loss"]
}

# --- Randomized search ---
dt_random = RandomizedSearchCV(
    dt_pipe,
    dt_param_dist,
    n_iter=20,              # number of random draws
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

# --- Fit ---
with parallel_backend("threading"):
    dt_random.fit(X_train, y_train)

# --- Evaluate ---
y_pred_test = dt_random.predict(X_test)
y_pred_proba = dt_random.predict_proba(X_test)[:, 1]

print("Decision Tree Best Params:", dt_random.best_params_)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_test))
print("Decision Tree AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# --- Store results ---
usdot_results.append({
    "Model": "Decision Tree (Random Search, class_weight=balanced)",
    "Best Params": dt_random.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})

# ---- Feature Importances ----
best_dt = dt_random.best_estimator_
dt_model = best_dt.named_steps["model"]

# Get feature names from the preprocessor
feature_names = best_dt.named_steps["preprocessor"].get_feature_names_out()

# Map importances back to feature names
importances = pd.Series(dt_model.feature_importances_, index=feature_names)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(
    x=importances_sorted.head(20),
    y=importances_sorted.head(20).index,
    palette="viridis"
)
plt.title("Top 20 Feature Importances - Decision Tree (Random Search)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

### Random Forecast Baseline

In [ ]:
# --- Step 1: Reduce dataset to 20% (stratified) ---
X_small, _, y_small, _ = train_test_split(
    X_mend,
    y_mend_binary_15,
    train_size=0.2,
    stratify=y_mend_binary_15,
    random_state=42
)

# --- Step 2: Train/test split (80/20 of reduced set) ---
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,
    stratify=y_small,
    random_state=42
)

# --- Pipeline: preprocessing + Random Forest ---
rf_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_mend),
    ("model", RandomForestClassifier(
        n_estimators=200,       # number of trees
        max_depth=None,         # let trees expand fully
        random_state=42,
        class_weight="balanced" # handle class imbalance
    ))
])

# --- Fit ---
with parallel_backend("threading"):
    rf_pipe.fit(X_train, y_train)

# --- Evaluate ---
y_pred_test = rf_pipe.predict(X_test)
y_pred_proba = rf_pipe.predict_proba(X_test)[:, 1]

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Forest AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# --- Feature Importances ---
rf_model = rf_pipe.named_steps["model"]
feature_names = rf_pipe.named_steps["preprocessor"].get_feature_names_out()

importances = pd.Series(rf_model.feature_importances_, index=feature_names)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(
    x=importances_sorted.head(20),
    y=importances_sorted.head(20).index,
    palette="viridis"
)
plt.title("Top 20 Feature Importances - Random Forest (Baseline)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# --- Store results ---
usdot_results.append({
    "Model": "Random Forest (Baseline, class_weight=balanced)",
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})

### RF w/ Random Search

In [ ]:
# --- Step 1: Reduce dataset to 20% (stratified) ---
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,
    stratify=y_usdot_binary_15,
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# --- Step 2: Train/test split (80/20 of reduced set) ---
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# --- Pipeline: preprocessing + Random Forest with class weights ---
rf_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", RandomForestClassifier(
        random_state=42,
        class_weight="balanced"   # <-- key change
    ))
])

# --- Parameter distributions for random search ---
rf_param_dist = {
    "model__n_estimators": randint(100, 500),
    "model__max_depth": [None, 5, 10, 20],
    "model__min_samples_split": randint(2, 10),
    "model__min_samples_leaf": randint(1, 5)
}

# --- Randomized search ---
rf_random = RandomizedSearchCV(
    rf_pipe,
    rf_param_dist,
    n_iter=20,
    cv=3,
    scoring="roc_auc",
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

# --- Fit ---
with parallel_backend("threading"):
    rf_random.fit(X_train, y_train)

# --- Evaluate ---
y_pred_test = rf_random.predict(X_test)
y_pred_proba = rf_random.predict_proba(X_test)[:, 1]

print("Random Forest Best Params:", rf_random.best_params_)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Forest AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# --- Store results ---
usdot_results.append({
    "Model": "Random Forest (Random Search, class_weight=balanced)",
    "Best Params": rf_random.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})

# ---- Feature Importances ----
best_rf = rf_random.best_estimator_
rf_model = best_rf.named_steps["model"]

# Get feature names from the preprocessor
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

# Map importances back to feature names
importances = pd.Series(rf_model.feature_importances_, index=feature_names)
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(
    x=importances_sorted.head(20),
    y=importances_sorted.head(20).index,
    palette="viridis"
)
plt.title("Top 20 Feature Importances - Random Forest (class_weight=balanced)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

### RF w/ Grid Search

In [ ]:
# Step 1: Reduce dataset to 20% of original size (stratified)
X_small, _, y_small, _ = train_test_split(
    X_usdot,
    y_usdot_binary_15,
    train_size=0.2,          # keep 20% of original
    stratify=y_usdot_binary_15, # preserve class balance
    random_state=42
)

print("Original size:", X_usdot.shape)
print("Reduced size:", X_small.shape)

# Step 2: Split reduced dataset into train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_small,
    y_small,
    test_size=0.2,           # 20% of reduced set → 16% train, 4% test of original
    stratify=y_small,
    random_state=42
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

# Pipeline: preprocessing + Random Forest
rf_pipe = Pipeline(steps=[
    ("preprocessor", reg_prep_usdot),
    ("model", RandomForestClassifier(random_state=42, class_weight="balanced"))
])

# Grid of hyperparameters
rf_param_grid = {
    "model__n_estimators": [50, 100],
    "model__max_depth": [5, 10, 25],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2]
}

# Grid search
rf_grid = GridSearchCV(
    rf_pipe,
    rf_param_grid,
    cv=2,
    scoring="roc_auc",
    n_jobs=-1
)

with parallel_backend("threading"):
    rf_grid.fit(X_train, y_train)

# Evaluate
y_pred_test = rf_grid.predict(X_test)
y_pred_proba = rf_grid.predict_proba(X_test)[:, 1]

print("Random Forest Best Params:", rf_grid.best_params_)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_test))
print("Random Forest AUC:", roc_auc_score(y_test, y_pred_proba))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

# Store results
usdot_results.append({
    "Model": "Random Forest (Grid Search, class_weight=balanced)",
    "Best Params": rf_grid.best_params_,
    "Test Accuracy": accuracy_score(y_test, y_pred_test),
    "Test AUC": roc_auc_score(y_test, y_pred_proba),
    "Balanced Accuracy": balanced_accuracy_score(y_test, y_pred_test)
})

importances = rf_grid.best_estimator_.named_steps["model"].feature_importances_
feature_names = rf_grid.best_estimator_.named_steps["preprocessor"].get_feature_names_out()
pd.Series(importances, index=feature_names).sort_values(ascending=False).head(20)

# Get the best fitted pipeline
best_rf = rf_grid.best_estimator_

# Extract the trained RandomForest model
rf_model = best_rf.named_steps["model"]

# Get feature names from the preprocessor
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

# Pair feature names with importances
importances = pd.Series(rf_model.feature_importances_, index=feature_names)

# Sort by importance
importances_sorted = importances.sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(10,6))
sns.barplot(x=importances_sorted.head(20), y=importances_sorted.head(20).index, palette="viridis")
plt.title("Top 20 Feature Importances - Random Forest")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

# --- ROC Curve ---
RocCurveDisplay.from_estimator(best_rf, X_test, y_test)
plt.title("ROC Curve - Random Forest (Grid Search)")
plt.show()

# --- Precision-Recall Curve ---
PrecisionRecallDisplay.from_estimator(best_rf, X_test, y_test)
plt.title("Precision-Recall Curve - Random Forest (Grid Search)")
plt.show()

# --- Confusion Matrix Heatmap ---
ConfusionMatrixDisplay.from_estimator(best_rf, X_test, y_test, cmap="Blues")
plt.title("Confusion Matrix - Random Forest (Grid Search)")
plt.show()

# Transform X_test with the preprocessor to get the actual feature matrix
X_test_transformed = best_rf.named_steps["preprocessor"].transform(X_test)

# Get the feature names aligned with the transformed matrix
feature_names = best_rf.named_steps["preprocessor"].get_feature_names_out()

print("X_test_transformed shape:", X_test_transformed.shape)
print("Number of feature names:", len(feature_names))
with parallel_backend("threading"):
    perm_importance = permutation_importance(
        best_rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )

# Align lengths
perm_sorted = pd.Series(
    perm_importance.importances_mean,
    index=feature_names[:len(perm_importance.importances_mean)]
).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=perm_sorted.head(20), y=perm_sorted.head(20).index, palette="magma")
plt.title("Top 20 Features - Permutation Importance")
plt.xlabel("Importance (mean decrease in score)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()



## Week 7 - Milestone 1

### Review Results

In [ ]:
# Convert list of dicts into DataFrame
usdot_results_df = pd.DataFrame(usdot_results)

# Sort by test_r2 (descending) or test_rmse (ascending)
usdot_results_df_sorted = usdot_results_df.sort_values(by="Test AUC", ascending=False)

print(usdot_results_df_sorted)

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=usdot_results_df, x="model", y="test_r2", palette="viridis")
plt.title("Model Comparison (Test R²)")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8,5))
sns.barplot(data=usdot_results_df, x="model", y="test_rmse", palette="magma")
plt.title("Model Comparison (Test RMSE)")
plt.xticks(rotation=45)
plt.show()